# Replication of "Evaluating Parameter-Efficient Finetuning Approaches for Pre-trained Models on the Financial Domain" paper results

This notebook is the code for replication of finetuning Bert Model for 4 Financial tasks and also using parameter efficient methods like Lora and Adapters.

## The structure of code is:
- Chapter 1: Sentiment Classification and Regression
  - 1.2 Lora method for each model type (Bert, FinBert and FlangBert)
  - 1.3 Adapter method for each models

- Chapter 3: News Headline Classification
  - 1.1 Bert Hyperparameter Search and training
  - 1.2 Lora method for each models
  - 1.3 Adapter method for each models

- Chapter 4: Named Entity Recognition
  - 1.1 Bert Hyperparameter Search and training
  - 1.2 Lora method for each models
  - 1.3 Adapter method for each models

## About Hyperparameter Search
We decreased the range of hyperparameter search for number of epochs and made constant for batch size depending on the task (we put the same batch size, which was the best in original paper).

- Full fine tuning:
  - epochs range: [3, 4, 5]
  - learning_rates range: [1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3]
  - batch size: depends on the task

- Lora fine tuning:
  - epochs range: [3, 4, 5]
  - learning_rates range: [1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3]
  - batch size: depends on the task

- Adapter fine tuning:
  - epochs range: [6, 9, 11]
  - learning_rates range: [1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3]
  - batch size: depends on the task

### For the convenience we left import libraries in each chapter as it is. Because at the first time it was done on seperate notebooks and we merged all them in one notebook. You can run which you want first, so it will not affect other chapters.

# Installing Necessary Libraries

Before going to one of the sections, install these necessary libraries

In [ ]:
#importing clear_output() function from notebook utilities
from IPython.display import clear_output

# After installing the adapters the will be like restarting, just press cancell and contunie to run. Because it install appropriate transformer library
!pip install adapters
!pip install datasets
!pip install torch
# For training process and looking at hyperparameters
!pip install huggingface_hub[hf_xet]
clear_output()
print('Done!')

Done!


# Sentiment Classification and Regression

## Loading and Preparing Datasets

In [ ]:
from IPython.display import clear_output
from datasets import load_dataset
from pprint import pprint
import json

# Load dataset

# Financial Phrasebank
FP_dataset = load_dataset("takala/financial_phrasebank", 'sentences_50agree', trust_remote_code=True)['train'] # 4846 instead of 4845
dataset_FP = [x for x in FP_dataset]

# FIQA files loading
with open('task1_headline_ABSA_train.json') as f:
    headlines = json.load(f)

with open('task1_post_ABSA_train.json') as f:
    posts = json.load(f)

# Clear the output
clear_output()

print('The dataset blueprint for Financial Phrasebank is: ')
print(dataset_FP)
print("===================")
print('The dataset blueprint for Question and Answering is: ')
print(len(headlines))
print(len(posts))
print("===================")

The dataset blueprint for Financial Phrasebank is: 
[{'sentence': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .', 'label': 1}, {'sentence': 'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .', 'label': 1}, {'sentence': 'The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .', 'label': 0}, {'sentence': 'With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .', 'label': 2}, {'sentence': "According to the company 's updated strategy for the years 200

In [ ]:
import random
from sklearn.model_selection import train_test_split

# Combine and process data
dataset_FIQA = []
for dataset in [headlines, posts]:
    for entry in dataset.values():
        sentence = entry['sentence']
        for info in entry['info']:
            target = info['target']
            score = float(info['sentiment_score'])
            input_text = f"{target}: {sentence}"
            dataset_FIQA.append({'sentence': input_text, 'label': score})

# Randomly select 1173 samples
random.seed(42)
dataset_FIQA = random.sample(dataset_FIQA, 1173)

## Functions for Tokenization and Classifiers

In [ ]:
from torch.utils.data import Dataset
import torch

class FP_FIQA_Dataset(Dataset):
    """
    PyTorch Dataset for Word in Context tokenization.
    """
    def __init__(self, task, dataset, tokenizer, max_length=128):
        # Initializing the instances of a class and assigning values from the parameter
        self.task = task
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        # Returning the count of the intences in the sample dataset for idx
        return len(self.dataset['sentence'])

    def __getitem__(self, idx):

        # Tokenizer for sentence
        tokens = self.tokenizer(
            self.dataset['sentence'][idx],
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt',
            return_offsets_mapping=True
        )

        return {
            'input_ids': tokens["input_ids"].squeeze(), # A 1 dimensional torch tensor representing input ids of the first sentence
            'attention_mask': tokens["attention_mask"].squeeze(), # A 1 dimensional torch tensor representing attention mask of the first sentence
            'label': torch.tensor(self.dataset['label'][idx], dtype=torch.float if self.task == "FIQA" else torch.long),# A scalar torch tensor representing the instance label
            }


In [ ]:
import torch.nn as nn
from peft import LoraConfig, get_peft_model, TaskType

class BertSentimentClassifier(nn.Module):

    def __init__(self, bert_model, lora):
        super().__init__()
        self.bert = bert_model

        self.classifier = nn.Linear(self.bert.config.hidden_size, 3)

    def forward(self, input_ids, attention_mask):
        # Get BERT outputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        # Get the CLS token representation (the first token)
        cls_output = outputs.last_hidden_state[:, 0, :]

        # Apply classifier
        logits = self.classifier(cls_output)

        return logits

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from adapters import AdapterConfig, AutoAdapterModel
from peft import LoraConfig, get_peft_model, TaskType

# Model for QA scoring (regression)
class BertRelevanceRegressor(nn.Module):
    def __init__(self, bert_model, lora):
        super().__init__()
        if lora:
            # Preparing model for LoRA
            peft_config = LoraConfig(
                task_type=TaskType.FEATURE_EXTRACTION,
                r=16,  # Rank
                lora_alpha=32,  # Scaling factor
                lora_dropout=0.05,
                target_modules=["query", "key", "value"]  # Modules to apply
            )

            # Wrapping BERT model with LoRA
            self.bert = get_peft_model(bert_model, peft_config)

            for name, param in self.bert.named_parameters():
              if "lora" not in name.lower():
                  param.requires_grad = False
        else:
            self.bert = bert_model

        self.regressor = nn.Sequential(
            nn.Linear(bert_model.config.hidden_size, 1),
            nn.Tanh()  # ensure output is in [-1, 1]
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        score = self.regressor(cls_output).squeeze(-1)
        return score

# Define the AdapterRegressor class
class AdapterRegressor(nn.Module):
    def __init__(self, adapter_name, model_name):
        super().__init__()
        # Load BERT with adapter support
        self.bert = AutoAdapterModel.from_pretrained(model_name)
        # Add adapter to the model
        self.bert.add_adapter(adapter_name, config="houlsby")
        # Activate the adapter
        self.bert.set_active_adapters(adapter_name)
        # Freeze all parameters except adapter
        self.bert.train_adapter(adapter_name)
        # Regressor head
        self.regressor = nn.Linear(768, 1)  # Assuming BERT base has 768 hidden size

    def forward(self, input_ids, attention_mask):
        # Set output_hidden_states=True to access hidden states
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True  # Request hidden states
        )

        # Properly handle the model outputs
        if hasattr(outputs, 'last_hidden_state'):
            # If the model returns last_hidden_state directly
            cls = outputs.last_hidden_state[:, 0, :]
        elif hasattr(outputs, 'hidden_states'):
            # If the model returns hidden_states
            cls = outputs.hidden_states[-1][:, 0, :]
        else:
            # Fallback: try to get from the first element of the tuple
            cls = outputs[0][:, 0, :]

        return torch.tanh(self.regressor(cls).squeeze(-1))

    def save_adapter(self, path, adapter_name):
        self.bert.save_adapter(path, adapter_name)


## Functions for Training and Hyperparameter Search

In [ ]:
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from datetime import datetime
import os
import json

# Function to train and evaluate model with specific hyperparameters
def train_and_evaluate_FP(model_name, train_loader, val_loader, test_loader, lr, epochs, device, lora, adapter):
    # Adapter configuration for Houlsby method
    adapter_config = AdapterConfig.load("houlsby")

    if adapter:
      # Initialize model for this specific run
      model = AutoAdapterModel.from_pretrained(model_name)
      # Add the Houlsby adapter
      adapter_name = "fp_adapter"
      model.add_adapter(adapter_name, config=adapter_config)
      model.add_classification_head("fp_houlsby", num_labels=3)
      # Activate adapter and freeze other parameters
      model.train_adapter(adapter_name)
      model.set_active_adapters(adapter_name)
    elif lora:
      lora_config = LoraConfig(
          task_type = TaskType.SEQ_CLS,
          r= 16,
          lora_alpha = 32,
          lora_dropout = 0.05,
          target_modules = ["query", "key", "value"],
          bias = "none"
      )
      model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
      trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
      print("number of trainable param: ", trainable_params)
      model = get_peft_model(model, lora_config)
      model.print_trainable_parameters()
    else:
      model = AutoModel.from_pretrained(model_name)
      model = BertSentimentClassifier(model, lora)

    model = model.to(device)

    if lora:
      optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), weight_decay=0.01)
    elif adapter:
      # Optimizer which focuses on adapter parameters
      optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=0.01)
    else:
      # Regular optimizer
      optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    criterion = nn.CrossEntropyLoss()

    # Track best performance
    best_val_acc = 0
    best_model_state = None
    epochs_since_improve = 0
    training_history = {
        'train_loss': [],
        'val_acc': [],
        'epoch': []
    }

    for epoch in range(epochs):
        # Training phase
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")

        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            if adapter:
              outputs = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels,
                    adapter_names=[adapter_name])
              loss = outputs.loss
            elif lora:
              outputs = model(input_ids, attention_mask, labels = labels)
              loss = outputs.loss
            else:
              outputs = model(input_ids, attention_mask)
              loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

        avg_train_loss = total_loss / len(train_loader)

        # Validation phase
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)

                if adapter:
                  outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels,
                        adapter_names=[adapter_name])
                  _, predicted = torch.max(outputs.logits, dim=1)
                elif lora:
                  outputs = model(input_ids, attention_mask, labels=labels)
                  _, predicted = torch.max(outputs.logits, dim=1)
                else:
                  outputs = model(input_ids, attention_mask)
                  _, predicted = torch.max(outputs.data, 1)

                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_acc = accuracy_score(all_labels, all_preds)

        # Save metrics for this epoch
        training_history['train_loss'].append(avg_train_loss)
        training_history['val_acc'].append(val_acc)
        training_history['epoch'].append(epoch + 1)


        print(f"Epoch {epoch+1}/{epochs} done!")
        print(f"Average training loss: {avg_train_loss:.4f}")
        print(f"Validation accuracy: {val_acc:.4f}")

        # Save best model
        # ——— Check for improvement ———
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
            epochs_since_improve = 0
            print(f"✓ New best model (val_acc: {best_val_acc:.4f})")
        else:
            epochs_since_improve += 1
            print(f"⟳ No improvement for {epochs_since_improve}/{3} epochs")
            if epochs_since_improve >= 3:
                print(f"⏹ Early stopping at epoch {epoch+1}")
                break
    if adapter:
        # Save best model
        model.save_adapter(os.path.join("./adapters", adapter_name), adapter_name)

    # Load best model state for testing
    if best_model_state:
        model.load_state_dict(best_model_state)

    # Test phase
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            if adapter:
              outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels,
                        adapter_names=[adapter_name])
              _, predicted = torch.max(outputs.logits, dim=1)

            elif lora:
              outputs = model(input_ids, attention_mask, labels=labels)
              _, predicted = torch.max(outputs.logits, dim=1)
            else:
              outputs = model(input_ids, attention_mask)
              _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    test_acc = accuracy_score(all_labels, all_preds)
    classification_rep = classification_report(all_labels, all_preds, output_dict=True)
    conf_matrix = confusion_matrix(all_labels, all_preds).tolist()

    best_result = {
        'learning_rate': lr,
        'epochs': epochs,
        'best_val_accuracy': best_val_acc,
        'test_accuracy': test_acc,
        'classification_report': classification_rep,
        'confusion_matrix': conf_matrix,
        'training_history': training_history,
        'best_model_state': best_model_state
    }

    # Return results
    return best_result

In [ ]:
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch.nn as nn
from adapters import AdapterConfig, AutoAdapterModel
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.metrics import  mean_squared_error, mean_absolute_error
from datetime import datetime
import os
import json

# Function to train and evaluate model with specific hyperparameters
def train_and_evaluate_FIQA(model_name, train_loader, val_loader, test_loader, lr, epochs, device, lora, adapter):
    # Adapter configuration for Houlsby method
    adapter_config = AdapterConfig.load("houlsby")
    if adapter:
      # Initialize model for this specific run
      model = AdapterRegressor(adapter_name="fiqa_adapter", model_name=model_name)
    else:
      model = AutoModel.from_pretrained(model_name)
      model = BertRelevanceRegressor(model, lora)
    model = model.to(device)

    if lora:
      # Optimizer which focused on LoRA and classifier parameters
      optimizer = torch.optim.AdamW([
          {'params': model.bert.parameters(), 'lr': lr},
          {'params': model.regressor.parameters(), 'lr': lr}
      ], weight_decay=0.01)
    elif adapter:
      # Optimizer which focuses on adapter parameters
      optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=0.02)
    else:
      # Regular optimizer
      optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    criterion = nn.MSELoss()
    best_val_mse = float('inf')
    best_model_state = None
    best_test_result = {}
    epochs_since_improve = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # Validation
        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)
                outputs = model(input_ids, attention_mask)
                val_preds.extend(outputs.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_mse = mean_squared_error(val_labels, val_preds)
        val_mae = mean_absolute_error(val_labels, val_preds)

        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val MSE = {val_mse:.4f}, Val MAE = {val_mae:.4f}")

        # 3) Early-stopping check
        if val_mse < best_val_mse:
            best_val_mse = val_mse
            best_model_state = model.state_dict().copy()
            epochs_since_improve = 0  # reset counter
        else:
            epochs_since_improve += 1
            print(f"  ↪ No improvement for {epochs_since_improve}/{3} epochs")
            if epochs_since_improve >= 3:
                print(f"Stopping early after {epoch+1} epochs.")
                break
    if adapter:
      # Save best model
      model.save_adapter(os.path.join("./adapters", "fiqa_adapter"), "fiqa_adapter")

    # Evaluate on test set with best model
    model.load_state_dict(best_model_state)
    model.eval()
    test_preds, test_labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask)
            test_preds.extend(outputs.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    test_mse = mean_squared_error(test_labels, test_preds)
    test_mae = mean_absolute_error(test_labels, test_preds)
    best_result = {
        'learning_rate': lr,
        'epochs': epochs,
        'best_val_mse': best_val_mse,
        'test_mse': test_mse,
        'test_mae': test_mae,
        'best_model_state': best_model_state
    }

    return best_result

In [ ]:
# Main function for grid search
import shutil
import torch
def hyperparameter_grid_search(model_name, lr, epochs, task, dataset, lora, adapter):

    learning_rates = lr
    num_epochs_list = epochs
    max_length = 128

    if task == 'FP' and not lora and not adapter:
      batch_size = 32
    else:
      batch_size = 16
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print(f"Using device: {device}")

    # Load pretrained tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load and prepare dataset

    train_data = dataset
    train, test = train_test_split(train_data, test_size=0.3, random_state=42)
    val, test = train_test_split(test, test_size=0.5, random_state=42)

    print(f"Dataset sizes - Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")

    # Convert to dictionaries with 'sentence' and 'label' keys
    train = {'sentence': [d['sentence'] for d in train], 'label': [d['label'] for d in train]}
    val = {'sentence': [d['sentence'] for d in val], 'label': [d['label'] for d in val]}
    test = {'sentence': [d['sentence'] for d in test], 'label': [d['label'] for d in test]}

    # Create datasets
    train_dataset = FP_FIQA_Dataset(task, train, tokenizer, max_length)
    val_dataset = FP_FIQA_Dataset(task, val, tokenizer, max_length)
    test_dataset = FP_FIQA_Dataset(task, test, tokenizer, max_length)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Create output directory for results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = f"bert_grid_search_{timestamp}"
    os.makedirs(results_dir, exist_ok=True)

    # Track all results
    all_results = []
    best_result = None
    # best_accuracy = 0
    best_metric = float('inf') if task == "FIQA" else 0

    # Grid search
    for lr in learning_rates:
        for epochs in num_epochs_list:
            run_name = f"lr_{lr}_epochs_{epochs}"
            print(f"\n{'='*50}\nStarting training with LR: {lr}, Epochs: {epochs}\n{'='*50}")

            if task == "FIQA":
                result = train_and_evaluate_FIQA(model_name, train_loader, val_loader, test_loader, lr, epochs, device, lora, adapter)
                metric = result['best_val_mse']
                is_better = metric < best_metric
            elif task == "FP":
                result = train_and_evaluate_FP(model_name, train_loader, val_loader, test_loader, lr, epochs, device, lora, adapter)
                metric = result['test_accuracy']
                is_better = metric > best_metric
            else:
                raise ValueError("Unsupported task type")

            # Save model
            if result['best_model_state']:
                torch.save(result['best_model_state'], f"{results_dir}/model_{run_name}.pt")
                model_state = result.pop('best_model_state')
                with open(f"{results_dir}/result_{run_name}.json", 'w') as f:
                    json.dump(result, f, indent=4)
                result['best_model_state'] = model_state

            if is_better:
                best_metric = metric
                best_result = result
                print(f"✓ New best overall model ({'MSE' if task == 'FIQA' else 'accuracy'}: {best_metric:.4f})")

            all_results.append(result)

    # Save all results (remove model state)
    serializable_results = []
    for result in all_results:
        res = result.copy()
        res.pop('best_model_state', None)
        serializable_results.append(res)

    with open(f"{results_dir}/all_results.json", 'w') as f:
        json.dump(serializable_results, f, indent=4)

    # Save best model info
    if best_result:
        print("\nBest Hyperparameters:")
        print(f"Learning Rate: {best_result['learning_rate']}")
        print(f"Epochs: {best_result['epochs']}")
        print(f"Batch Size: {batch_size}")
        if task == "FIQA":
            print(f"Validation MSE: {best_result['best_val_mse']:.4f}")
            print(f"Test MSE: {best_result['test_mse']:.4f}")
        else:
            print(f"Validation Accuracy: {best_result['best_val_accuracy']:.4f}")
            print(f"Test Accuracy: {best_result['test_accuracy']:.4f}")

        model_path = f"{results_dir}/model_lr_{best_result['learning_rate']}_epochs_{best_result['epochs']}.pt"
        if os.path.exists(model_path):
            shutil.copy(model_path, f"{results_dir}/best_model.pt")

    print(f"\nAll results saved in the '{results_dir}' directory.")
    return best_result

## Hyperparameter Search and Bert Model Train (Financial Phrasebank and FIQA)

### Bert on Financial Phrasebank

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'bert-base-uncased'
lr = [1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3]
epochs = [3,4,5]
task = 'FP'
dataset = dataset_FP
lora = False
adapter = False

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

this works
Using device: cuda
Dataset sizes - Train: 3392, Val: 727, Test: 727

Starting training with LR: 1e-06, Epochs: 3


Epoch 1/3 [Val]: 100%|██████████| 23/23 [00:02<00:00,  9.42it/s]


Epoch 1/3 done!
Average training loss: 0.9598
Validation accuracy: 0.6121
✓ New best model (val_acc: 0.6121)


Epoch 2/3 [Val]: 100%|██████████| 23/23 [00:02<00:00,  9.63it/s]


Epoch 2/3 done!
Average training loss: 0.8221
Validation accuracy: 0.6740
✓ New best model (val_acc: 0.6740)


...

Epoch 3/5 done!
Average training loss: 0.9357
Validation accuracy: 0.5970
⟳ No improvement for 2/3 epochs


Epoch 4/5 [Val]: 100%|██████████| 23/23 [00:02<00:00, 10.09it/s]


Epoch 4/5 done!
Average training loss: 0.9366
Validation accuracy: 0.5970
⟳ No improvement for 3/3 epochs
⏹ Early stopping at epoch 4


Testing: 100%|██████████| 23/23 [00:02<00:00, 10.11it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r


Best Hyperparameters:
Learning Rate: 5e-05
Epochs: 3
Batch Size: 32
Validation Accuracy: 0.8693
Test Accuracy: 0.8624

All results saved in the 'bert_grid_search_20250503_132808' directory.


### Bert + LoRA on Financial Phrasebank

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'bert-base-uncased'
lr = [1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3]
epochs = [3,4,5]
task = 'FP'
dataset = dataset_FP
lora = True
adapter = False

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 3392, Val: 727, Test: 727

Starting training with LR: 1e-06, Epochs: 3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


number of trainable param:  109484547
trainable params: 887,043 || all params: 110,371,590 || trainable%: 0.8037


Epoch 1/3 [Val]: 100%|██████████| 46/46 [00:02<00:00, 18.25it/s]


Epoch 1/3 done!
Average training loss: 0.6104
Validation accuracy: 0.8336
✓ New best model (val_acc: 0.8336)


...

Epoch 4/5 done!
Average training loss: 0.3430
Validation accuracy: 0.8514
⟳ No improvement for 1/3 epochs


Epoch 5/5 [Val]: 100%|██████████| 46/46 [00:02<00:00, 17.68it/s]


Epoch 5/5 done!
Average training loss: 0.3112
Validation accuracy: 0.7950
⟳ No improvement for 2/3 epochs


Testing: 100%|██████████| 46/46 [00:02<00:00, 17.51it/s]



Best Hyperparameters:
Learning Rate: 0.001
Epochs: 3
Batch Size: 16
Validation Accuracy: 0.8473
Test Accuracy: 0.8501

All results saved in the 'bert_grid_search_20250503_143836' directory.


### Bert + AdapterH on Financial Phrasebank

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'bert-base-uncased'
lr = [1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3]
epochs = [6,9,11]
task = 'FP'
dataset = dataset_FP
lora = False
adapter = True

# Making folder
if adapter:
  os.makedirs("./adapters", exist_ok=True)

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 3392, Val: 727, Test: 727

Starting training with LR: 1e-06, Epochs: 6


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/6 [Val]: 100%|██████████| 46/46 [00:02<00:00, 16.88it/s]


Epoch 1/6 done!
Average training loss: 1.0695
Validation accuracy: 0.6011
✓ New best model (val_acc: 0.6011)


Epoch 2/6 [Val]: 100%|██████████| 46/46 [00:02<00:00, 17.39it/s]


Epoch 2/6 done!
Average training loss: 0.9834
Validation accuracy: 0.5970
⟳ No improvement for 1/3 epochs


...

Epoch 7/11 done!
Average training loss: 0.1172
Validation accuracy: 0.8432
⟳ No improvement for 2/3 epochs


Epoch 8/11 [Val]: 100%|██████████| 46/46 [00:02<00:00, 17.38it/s]


Epoch 8/11 done!
Average training loss: 0.0946
Validation accuracy: 0.8363
⟳ No improvement for 3/3 epochs
⏹ Early stopping at epoch 8


Testing: 100%|██████████| 46/46 [00:02<00:00, 16.92it/s]



Best Hyperparameters:
Learning Rate: 0.0001
Epochs: 9
Batch Size: 16
Validation Accuracy: 0.8556
Test Accuracy: 0.8583

All results saved in the 'bert_grid_search_20250503_153905' directory.


### Bert on FIQA

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'bert-base-uncased'
task = 'FIQA'
dataset = dataset_FIQA
lr = [1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3]
epochs = [3,4,5]
lora = False
adapter = False

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch 2: Train Loss = 0.9372, Val MSE = 1.0584, Val MAE = 0.9453
  ↪ No improvement for 1/3 epochs


Epoch 3/5 [Train]: 100%|██████████| 52/52 [00:08<00:00,  5.95it/s]


Epoch 3: Train Loss = 0.9399, Val MSE = 1.0584, Val MAE = 0.9453
  ↪ No improvement for 2/3 epochs


Epoch 4/5 [Train]: 100%|██████████| 52/52 [00:08<00:00,  5.95it/s]


Epoch 4: Train Loss = 0.9299, Val MSE = 1.0584, Val MAE = 0.9453
  ↪ No improvement for 3/3 epochs
Stopping early after 4 epochs.

Best Hyperparameters:
Learning Rate: 0.0001
Epochs: 5
Batch Size: 16
Validation MSE: 0.0573
Test MSE: 0.0773

All results saved in the 'bert_grid_search_20250503_142242' directory.


### Bert + LoRA on FIQA

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'bert-base-uncased'
lr = [1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3]
epochs = [3,4,5]
task = 'FIQA'
dataset = dataset_FIQA
lora = True
adapter = False

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch 3: Train Loss = 0.0523, Val MSE = 0.0710, Val MAE = 0.1978


Epoch 4/5 [Train]: 100%|██████████| 52/52 [00:05<00:00,  8.98it/s]


Epoch 4: Train Loss = 0.0360, Val MSE = 0.0958, Val MAE = 0.2238
  ↪ No improvement for 1/3 epochs


Epoch 5/5 [Train]: 100%|██████████| 52/52 [00:05<00:00,  8.99it/s]


Epoch 5: Train Loss = 0.0257, Val MSE = 0.0735, Val MAE = 0.2000
  ↪ No improvement for 2/3 epochs

Best Hyperparameters:
Learning Rate: 0.001
Epochs: 4
Batch Size: 16
Validation MSE: 0.0696
Test MSE: 0.1056

All results saved in the 'bert_grid_search_20250503_152352' directory.


### Bert + AdapterH on FIQA

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'bert-base-uncased'
lr = [1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3]
epochs = [6,9,11]
task = 'FIQA'
dataset = dataset_FIQA
lora = False
adapter = True

# Making folder
if adapter:
  os.makedirs("./adapters", exist_ok=True)

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch 4: Train Loss = 0.0370, Val MSE = 0.0873, Val MAE = 0.2047
  ↪ No improvement for 1/3 epochs


Epoch 5/11 [Train]: 100%|██████████| 52/52 [00:06<00:00,  8.23it/s]


Epoch 5: Train Loss = 0.0264, Val MSE = 0.0868, Val MAE = 0.2054
  ↪ No improvement for 2/3 epochs


Epoch 6/11 [Train]: 100%|██████████| 52/52 [00:06<00:00,  8.20it/s]


Epoch 6: Train Loss = 0.0189, Val MSE = 0.0876, Val MAE = 0.2080
  ↪ No improvement for 3/3 epochs
Stopping early after 6 epochs.
✓ New best overall model (MSE: 0.0778)

Best Hyperparameters:
Learning Rate: 0.001
Epochs: 11
Batch Size: 16
Validation MSE: 0.0778
Test MSE: 0.0940

All results saved in the 'bert_grid_search_20250503_165741' directory.


## FinBert on Financial Phrasebank

FinBert Finetuning

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'yiyanghkust/finbert-pretrain'
lr = [5e-5]
epochs = [3]
task = 'FP'
dataset = dataset_FP
lora = False
adapter = False

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 3392, Val: 727, Test: 727

Starting training with LR: 5e-05, Epochs: 3


Epoch 1/3 [Val]: 100%|██████████| 23/23 [00:02<00:00, 11.25it/s]


Epoch 1/3 done!
Average training loss: 0.5066
Validation accuracy: 0.8446
✓ New best model (val_acc: 0.8446)


Epoch 2/3 [Val]: 100%|██████████| 23/23 [00:02<00:00, 11.27it/s]


Epoch 2/3 done!
Average training loss: 0.2123
Validation accuracy: 0.7923
⟳ No improvement for 1/3 epochs


Epoch 3/3 [Val]: 100%|██████████| 23/23 [00:02<00:00, 11.32it/s]


Epoch 3/3 done!
Average training loss: 0.0827
Validation accuracy: 0.8459
✓ New best model (val_acc: 0.8459)


Testing: 100%|██████████| 23/23 [00:02<00:00, 11.36it/s]


✓ New best overall model (accuracy: 0.8556)

Best Hyperparameters:
Learning Rate: 5e-05
Epochs: 3
Batch Size: 32
Validation Accuracy: 0.8459
Test Accuracy: 0.8556

All results saved in the 'bert_grid_search_20250503_172126' directory.


FinBert + LoRA

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'yiyanghkust/finbert-pretrain'
lr = [1e-3]
epochs = [3]
task = 'FP'
dataset = dataset_FP
lora = True
adapter = False

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 3392, Val: 727, Test: 727

Starting training with LR: 0.001, Epochs: 3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


number of trainable param:  109754115
trainable params: 887,043 || all params: 110,641,158 || trainable%: 0.8017


Epoch 1/3 [Val]: 100%|██████████| 46/46 [00:02<00:00, 19.96it/s]


Epoch 1/3 done!
Average training loss: 0.5436
Validation accuracy: 0.8226
✓ New best model (val_acc: 0.8226)


...

Epoch 2/3 done!
Average training loss: 0.3851
Validation accuracy: 0.8184
⟳ No improvement for 1/3 epochs


Epoch 3/3 [Val]: 100%|██████████| 46/46 [00:02<00:00, 20.20it/s]


Epoch 3/3 done!
Average training loss: 0.3092
Validation accuracy: 0.8418
✓ New best model (val_acc: 0.8418)


Testing: 100%|██████████| 46/46 [00:02<00:00, 20.16it/s]


✓ New best overall model (accuracy: 0.8294)

Best Hyperparameters:
Learning Rate: 0.001
Epochs: 3
Batch Size: 16
Validation Accuracy: 0.8418
Test Accuracy: 0.8294

All results saved in the 'bert_grid_search_20250503_173501' directory.


FinBert + AdapterH

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'yiyanghkust/finbert-pretrain'
lr = [1e-4]
epochs = [9]
task = 'FP'
dataset = dataset_FP
lora = False
adapter = True

# Making folder
if adapter:
  os.makedirs("./adapters", exist_ok=True)

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 3392, Val: 727, Test: 727

Starting training with LR: 0.0001, Epochs: 9


Some weights of BertAdapterModel were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/9 [Val]: 100%|██████████| 46/46 [00:02<00:00, 19.51it/s]


Epoch 1/9 done!
Average training loss: 0.5815
Validation accuracy: 0.8116
✓ New best model (val_acc: 0.8116)


Epoch 2/9 [Val]: 100%|██████████| 46/46 [00:02<00:00, 19.62it/s]


Epoch 2/9 done!
Average training loss: 0.4003
Validation accuracy: 0.8239
✓ New best model (val_acc: 0.8239)


...

Epoch 7/9 done!
Average training loss: 0.1826
Validation accuracy: 0.8583
⟳ No improvement for 2/3 epochs


Epoch 8/9 [Val]: 100%|██████████| 46/46 [00:02<00:00, 19.64it/s]


Epoch 8/9 done!
Average training loss: 0.1545
Validation accuracy: 0.8556
⟳ No improvement for 3/3 epochs
⏹ Early stopping at epoch 8


Testing: 100%|██████████| 46/46 [00:02<00:00, 19.68it/s]


✓ New best overall model (accuracy: 0.8446)

Best Hyperparameters:
Learning Rate: 0.0001
Epochs: 9
Batch Size: 16
Validation Accuracy: 0.8597
Test Accuracy: 0.8446

All results saved in the 'bert_grid_search_20250503_172627' directory.


## FinBert on FIQA

FinBert Finetuning

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'yiyanghkust/finbert-pretrain'
lr = [1e-4]
epochs = [5]
task = 'FIQA'
dataset = dataset_FIQA
lora = False
adapter = False

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 821, Val: 176, Test: 176

Starting training with LR: 0.0001, Epochs: 5


Epoch 1/5 [Train]: 100%|██████████| 52/52 [00:08<00:00,  6.16it/s]


Epoch 1: Train Loss = 0.6293, Val MSE = 0.3072, Val MAE = 0.4264


Epoch 2/5 [Train]: 100%|██████████| 52/52 [00:08<00:00,  6.08it/s]


Epoch 2: Train Loss = 0.2100, Val MSE = 0.2210, Val MAE = 0.4104


...

Epoch 3: Train Loss = 0.1603, Val MSE = 0.3007, Val MAE = 0.4394
  ↪ No improvement for 1/3 epochs


Epoch 4/5 [Train]: 100%|██████████| 52/52 [00:08<00:00,  6.03it/s]


Epoch 4: Train Loss = 0.1288, Val MSE = 0.1044, Val MAE = 0.2456


Epoch 5/5 [Train]: 100%|██████████| 52/52 [00:08<00:00,  6.08it/s]


Epoch 5: Train Loss = 0.0679, Val MSE = 0.0901, Val MAE = 0.2309
✓ New best overall model (MSE: 0.0901)

Best Hyperparameters:
Learning Rate: 0.0001
Epochs: 5
Batch Size: 16
Validation MSE: 0.0901
Test MSE: 0.1180

All results saved in the 'bert_grid_search_20250503_172350' directory.


FinBert + LoRA

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'yiyanghkust/finbert-pretrain'
lr = [1e-3]
epochs = [4]
task = 'FIQA'
dataset = dataset_FIQA
lora = True
adapter = False

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 821, Val: 176, Test: 176

Starting training with LR: 0.001, Epochs: 4


Epoch 1/4 [Train]: 100%|██████████| 52/52 [00:05<00:00,  9.45it/s]


Epoch 1: Train Loss = 0.1704, Val MSE = 0.1222, Val MAE = 0.2798


Epoch 2/4 [Train]: 100%|██████████| 52/52 [00:05<00:00,  9.41it/s]


Epoch 2: Train Loss = 0.0958, Val MSE = 0.1141, Val MAE = 0.2575


Epoch 3/4 [Train]: 100%|██████████| 52/52 [00:05<00:00,  9.42it/s]


Epoch 3: Train Loss = 0.0521, Val MSE = 0.1068, Val MAE = 0.2518


Epoch 4/4 [Train]: 100%|██████████| 52/52 [00:05<00:00,  9.41it/s]


Epoch 4: Train Loss = 0.0396, Val MSE = 0.0898, Val MAE = 0.2301
✓ New best overall model (MSE: 0.0898)

Best Hyperparameters:
Learning Rate: 0.001
Epochs: 4
Batch Size: 16
Validation MSE: 0.0898
Test MSE: 0.0965

All results saved in the 'bert_grid_search_20250503_172559' directory.


FinBert + AdapterH

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'yiyanghkust/finbert-pretrain'
lr = [1e-3]
epochs = [11]
task = 'FIQA'
dataset = dataset_FIQA
lora = False
adapter = True

# Making folder
if adapter:
  os.makedirs("./adapters", exist_ok=True)

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 821, Val: 176, Test: 176

Starting training with LR: 0.001, Epochs: 11


Some weights of BertAdapterModel were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/11 [Train]: 100%|██████████| 52/52 [00:06<00:00,  8.45it/s]


Epoch 1: Train Loss = 0.1996, Val MSE = 0.1194, Val MAE = 0.2812


Epoch 2/11 [Train]: 100%|██████████| 52/52 [00:06<00:00,  8.42it/s]


Epoch 2: Train Loss = 0.1006, Val MSE = 0.1224, Val MAE = 0.2721
  ↪ No improvement for 1/3 epochs


...

Epoch 8: Train Loss = 0.0245, Val MSE = 0.0928, Val MAE = 0.2358
  ↪ No improvement for 1/3 epochs


Epoch 9/11 [Train]: 100%|██████████| 52/52 [00:06<00:00,  8.51it/s]


Epoch 9: Train Loss = 0.0208, Val MSE = 0.0775, Val MAE = 0.2097
  ↪ No improvement for 2/3 epochs


Epoch 10/11 [Train]: 100%|██████████| 52/52 [00:06<00:00,  8.49it/s]


Epoch 10: Train Loss = 0.0176, Val MSE = 0.0804, Val MAE = 0.2164
  ↪ No improvement for 3/3 epochs
Stopping early after 10 epochs.
✓ New best overall model (MSE: 0.0738)

Best Hyperparameters:
Learning Rate: 0.001
Epochs: 11
Batch Size: 16
Validation MSE: 0.0738
Test MSE: 0.0752

All results saved in the 'bert_grid_search_20250504_145511' directory.


## FlangBert on Financial Phrasebank

FlangBert Finetuning

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'SALT-NLP/FLANG-BERT'
lr = [5e-5]
epochs = [3]
task = 'FP'
dataset = dataset_FP
lora = False
adapter = False

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Dataset sizes - Train: 3392, Val: 727, Test: 727

Starting training with LR: 5e-05, Epochs: 3


config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

...

Epoch 2/3 done!
Average training loss: 0.2487
Validation accuracy: 0.8432
⟳ No improvement for 1/3 epochs


Epoch 3/3 [Val]: 100%|██████████| 23/23 [00:02<00:00,  9.48it/s]


Epoch 3/3 done!
Average training loss: 0.1277
Validation accuracy: 0.8418
⟳ No improvement for 2/3 epochs


Testing: 100%|██████████| 23/23 [00:02<00:00,  9.50it/s]


✓ New best overall model (accuracy: 0.8501)

Best Hyperparameters:
Learning Rate: 5e-05
Epochs: 3
Batch Size: 32
Validation Accuracy: 0.8459
Test Accuracy: 0.8501

All results saved in the 'bert_grid_search_20250503_180211' directory.


FlangBert + Lora

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'SALT-NLP/FLANG-BERT'
lr = [1e-3]
epochs = [3]
task = 'FP'
dataset = dataset_FP
lora = True
adapter = False

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 3392, Val: 727, Test: 727

Starting training with LR: 0.001, Epochs: 3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at SALT-NLP/FLANG-BERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


number of trainable param:  109484547
trainable params: 887,043 || all params: 110,371,590 || trainable%: 0.8037


Epoch 1/3 [Val]: 100%|██████████| 46/46 [00:02<00:00, 18.14it/s]


Epoch 1/3 done!
Average training loss: 0.6216
Validation accuracy: 0.8514
✓ New best model (val_acc: 0.8514)


...

Epoch 2/3 done!
Average training loss: 0.3664
Validation accuracy: 0.8391
⟳ No improvement for 1/3 epochs


Epoch 3/3 [Val]: 100%|██████████| 46/46 [00:02<00:00, 18.22it/s]


Epoch 3/3 done!
Average training loss: 0.3222
Validation accuracy: 0.8377
⟳ No improvement for 2/3 epochs


Testing: 100%|██████████| 46/46 [00:02<00:00, 18.22it/s]


✓ New best overall model (accuracy: 0.8212)

Best Hyperparameters:
Learning Rate: 0.001
Epochs: 3
Batch Size: 16
Validation Accuracy: 0.8514
Test Accuracy: 0.8212

All results saved in the 'bert_grid_search_20250503_180406' directory.


FlangBert + AdapterH

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'SALT-NLP/FLANG-BERT'
lr = [1e-4]
epochs = [9]
task = 'FP'
dataset = dataset_FP
lora = False
adapter = True

# Making folder
if adapter:
  os.makedirs("./adapters", exist_ok=True)

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 3392, Val: 727, Test: 727

Starting training with LR: 0.0001, Epochs: 9


Some weights of BertAdapterModel were not initialized from the model checkpoint at SALT-NLP/FLANG-BERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/9 [Val]: 100%|██████████| 46/46 [00:02<00:00, 16.92it/s]


Epoch 1/9 done!
Average training loss: 0.6872
Validation accuracy: 0.8116
✓ New best model (val_acc: 0.8116)


Epoch 2/9 [Val]: 100%|██████████| 46/46 [00:02<00:00, 16.95it/s]


Epoch 2/9 done!
Average training loss: 0.4165
Validation accuracy: 0.8666
✓ New best model (val_acc: 0.8666)


...

Epoch 6/9 done!
Average training loss: 0.2176
Validation accuracy: 0.8638
⟳ No improvement for 2/3 epochs


Epoch 7/9 [Val]: 100%|██████████| 46/46 [00:02<00:00, 16.98it/s]


Epoch 7/9 done!
Average training loss: 0.1962
Validation accuracy: 0.8611
⟳ No improvement for 3/3 epochs
⏹ Early stopping at epoch 7


Testing: 100%|██████████| 46/46 [00:02<00:00, 16.93it/s]


✓ New best overall model (accuracy: 0.8542)

Best Hyperparameters:
Learning Rate: 0.0001
Epochs: 9
Batch Size: 16
Validation Accuracy: 0.8817
Test Accuracy: 0.8542

All results saved in the 'bert_grid_search_20250504_150851' directory.


## FlangBert on FIQA

FlangBert Finetuning

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'SALT-NLP/FLANG-BERT'
lr = [1e-4]
epochs = [5]
task = 'FIQA'
dataset = dataset_FIQA
lora = False
adapter = False

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 821, Val: 176, Test: 176

Starting training with LR: 0.0001, Epochs: 5


Some weights of BertModel were not initialized from the model checkpoint at SALT-NLP/FLANG-BERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/5 [Train]: 100%|██████████| 52/52 [00:08<00:00,  5.95it/s]


Epoch 1: Train Loss = 0.1846, Val MSE = 0.1658, Val MAE = 0.3584


Epoch 2/5 [Train]: 100%|██████████| 52/52 [00:08<00:00,  6.04it/s]


Epoch 2: Train Loss = 0.1466, Val MSE = 0.0805, Val MAE = 0.2065


...

Epoch 3: Train Loss = 0.0812, Val MSE = 0.0770, Val MAE = 0.1936


Epoch 4/5 [Train]: 100%|██████████| 52/52 [00:08<00:00,  6.02it/s]


Epoch 4: Train Loss = 0.0337, Val MSE = 0.0946, Val MAE = 0.2227
  ↪ No improvement for 1/3 epochs


Epoch 5/5 [Train]: 100%|██████████| 52/52 [00:08<00:00,  6.03it/s]


Epoch 5: Train Loss = 0.0201, Val MSE = 0.0736, Val MAE = 0.1945
✓ New best overall model (MSE: 0.0736)

Best Hyperparameters:
Learning Rate: 0.0001
Epochs: 5
Batch Size: 16
Validation MSE: 0.0736
Test MSE: 0.0733

All results saved in the 'bert_grid_search_20250503_180926' directory.


FlangBert + Lora

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'SALT-NLP/FLANG-BERT'
lr = [1e-3]
epochs = [4]
task = 'FIQA'
dataset = dataset_FIQA
lora = True
adapter = False

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 821, Val: 176, Test: 176

Starting training with LR: 0.001, Epochs: 4


Some weights of BertModel were not initialized from the model checkpoint at SALT-NLP/FLANG-BERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/4 [Train]: 100%|██████████| 52/52 [00:05<00:00,  8.96it/s]


Epoch 1: Train Loss = 0.1672, Val MSE = 0.1676, Val MAE = 0.3537


Epoch 2/4 [Train]: 100%|██████████| 52/52 [00:05<00:00,  8.90it/s]


Epoch 2: Train Loss = 0.1290, Val MSE = 0.0867, Val MAE = 0.2162


Epoch 3/4 [Train]: 100%|██████████| 52/52 [00:05<00:00,  8.87it/s]


Epoch 3: Train Loss = 0.0666, Val MSE = 0.0859, Val MAE = 0.2117


Epoch 4/4 [Train]: 100%|██████████| 52/52 [00:05<00:00,  8.76it/s]


Epoch 4: Train Loss = 0.0358, Val MSE = 0.0618, Val MAE = 0.1782
✓ New best overall model (MSE: 0.0618)

Best Hyperparameters:
Learning Rate: 0.001
Epochs: 4
Batch Size: 16
Validation MSE: 0.0618
Test MSE: 0.0833

All results saved in the 'bert_grid_search_20250505_103931' directory.


FlangBert + AdapterH

In [ ]:
# Defining parameters for hyperparameter_grid_search function
model = 'SALT-NLP/FLANG-BERT'
lr = [1e-3]
epochs = [11]
task = 'FIQA'
dataset = dataset_FIQA
lora = False
adapter = True

# Making folder
if adapter:
  os.makedirs("./adapters", exist_ok=True)

best_params = hyperparameter_grid_search(model, lr, epochs, task, dataset,lora=lora, adapter=adapter)

Using device: cuda
Dataset sizes - Train: 821, Val: 176, Test: 176

Starting training with LR: 0.001, Epochs: 11


Some weights of BertAdapterModel were not initialized from the model checkpoint at SALT-NLP/FLANG-BERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/11 [Train]: 100%|██████████| 52/52 [00:06<00:00,  8.38it/s]


Epoch 1: Train Loss = 0.1325, Val MSE = 0.1084, Val MAE = 0.2788


Epoch 2/11 [Train]: 100%|██████████| 52/52 [00:06<00:00,  8.36it/s]


Epoch 2: Train Loss = 0.0821, Val MSE = 0.0779, Val MAE = 0.1942


...

Epoch 4: Train Loss = 0.0350, Val MSE = 0.0702, Val MAE = 0.1900
  ↪ No improvement for 1/3 epochs


Epoch 5/11 [Train]: 100%|██████████| 52/52 [00:06<00:00,  8.43it/s]


Epoch 5: Train Loss = 0.0264, Val MSE = 0.0685, Val MAE = 0.1790
  ↪ No improvement for 2/3 epochs


Epoch 6/11 [Train]: 100%|██████████| 52/52 [00:06<00:00,  8.43it/s]


Epoch 6: Train Loss = 0.0200, Val MSE = 0.0769, Val MAE = 0.1903
  ↪ No improvement for 3/3 epochs
Stopping early after 6 epochs.
✓ New best overall model (MSE: 0.0663)

Best Hyperparameters:
Learning Rate: 0.001
Epochs: 11
Batch Size: 16
Validation MSE: 0.0663
Test MSE: 0.0702

All results saved in the 'bert_grid_search_20250503_181211' directory.


# News Headline Classification

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.optim as optim
from sklearn.metrics import f1_score
from tqdm import tqdm
#LoRA & Adapter imports
from peft import get_peft_model, LoraConfig, TaskType
import adapters
from adapters import AdapterConfig


## Loading Data

In [ ]:
import kagglehub
import shutil
from IPython.display import clear_output
# Download it from kaggle datasets and place it in google colab
path = kagglehub.dataset_download("daittan/gold-commodity-news-and-dimensions")

# Move everything from the dataset folder to /content
destination = "/content/"
for filename in os.listdir(path):
    full_file_path = os.path.join(path, filename)
    if os.path.isfile(full_file_path):
        shutil.copy(full_file_path, destination)

# Moved them in content folder for easy access
clear_output()
print("Path to dataset files:", path)
print(f"Files moved to {destination}")

Path to dataset files: /root/.cache/kagglehub/datasets/daittan/gold-commodity-news-and-dimensions/versions/1
Files moved to /content/


## Data Preprocessing

Function to preprocess the data with appropriate tokenizer for model type

In [ ]:
def prepare_data(model_name, dataset_path, batch_size, seed=42):
    label_names = [
        'Price or Not', 'Direction Up', 'Direction Constant', 'Direction Down',
        'PastPrice', 'FuturePrice', 'PastNews', 'FutureNews', 'Asset Comparision'
    ]
    df = pd.read_csv(dataset_path)
    # import tokenizer depending on model name
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    tokens = tokenizer(df['News'].astype(str).tolist(), padding=True, truncation=True, return_tensors='pt')
    input_ids = tokens['input_ids']
    attention_masks = tokens['attention_mask']
    labels_tensor = torch.FloatTensor(df[label_names].values.astype(int))

    dataset = TensorDataset(input_ids, attention_masks, labels_tensor)

    torch.manual_seed(seed)
    total = len(dataset)
    test_size = int(0.2 * total)
    val_size = int(0.1 * total)
    train_size = total - val_size - test_size
    train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

    dataloaders = {
        'train': DataLoader(train_ds, batch_size=batch_size, shuffle=True),
        'val':   DataLoader(val_ds, batch_size=batch_size, shuffle=False),
        'test':  DataLoader(test_ds, batch_size=batch_size, shuffle=False)
    }

    return dataloaders, label_names


## Training function

In [ ]:
# this function is for computing weighted average across classes
def compute_per_class_f1(y_true, y_pred):
    scores = []
    for i in range(y_true.shape[1]):
        f1 = f1_score(y_true[:, i], y_pred[:, i], average='weighted', zero_division=0)
        scores.append(f1)
    return np.array(scores), np.mean(scores)

In [ ]:
def train_model( model_name, method, learning_rate, num_epochs, batch_size, seed, device, label_names, dataloaders):
    # Setup
    torch.manual_seed(seed)
    np.random.seed(seed)
    num_labels = len(label_names)
    patience = 3
    eps = 1e-2
    threshold = 0.5

    # loading the model
    base_model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
        problem_type='multi_label_classification'
    )

    # depending on method, apply appropriate method for the model
    if method == 'lora':
        lora_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            inference_mode=False,
            r=8, lora_alpha=16, lora_dropout=0.05,
            target_modules=["query", "key", "value"],
            bias="none"
        )
        model = get_peft_model(base_model, lora_config)

        # printing number of trainable parameter, to check does lora method applied
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Trainable parameters in Lora case: {trainable_params:,} / {total_params:,}",f"({100 * trainable_params / total_params:.2f}%)")
    elif method == 'adapter':
        adapters.init(base_model)
        adapter_config = AdapterConfig.load("houlsby", reduction_factor=16)
        adapter_name = "finance_adapter"
        base_model.add_adapter(adapter_name, config=adapter_config)
        base_model.train_adapter(adapter_name)
        base_model.set_active_adapters(adapter_name)
        model = base_model

        # printing number of trainable parameter, to check does adapter method applied
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Trainable parameters in Adapter case: {trainable_params:,} / {total_params:,}",f"({100 * trainable_params / total_params:.2f}%)")
    else:
        model = base_model
        # printing number of trainable parameter, just for checking
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Trainable parameters in Bert case: {trainable_params:,} / {total_params:,}",f"({100 * trainable_params / total_params:.2f}%)")

    model.to(device)
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)


    best_val_loss = float('inf')
    best_val_f1 = 0.0
    early_stop_counter = 0

    # Actual training loop
    for epoch in range(1, num_epochs + 1):
        if early_stop_counter >= patience:
            print("  ==> Early stopping triggered.")
            break

        print(f"\n=> Epoch {epoch}/{num_epochs}")
        model.train()
        train_loss = 0.0
        train_loop = tqdm(dataloaders['train'], desc="Training", leave=False)
        for ids, masks, lbls in train_loop:
            ids, masks, lbls = ids.to(device), masks.to(device), lbls.to(device)
            optimizer.zero_grad()
            loss = model(input_ids=ids, attention_mask=masks, labels=lbls).loss
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * ids.size(0)
            train_loop.set_postfix(loss=loss.item())
        train_loss /= len(dataloaders['train'].dataset)

        # Validation
        model.eval()
        val_loss, all_true, all_pred = 0.0, [], []
        val_loop = tqdm(dataloaders['val'], desc="Validating", leave=False)
        with torch.no_grad():
            for ids, masks, lbls in val_loop:
                ids, masks, lbls = ids.to(device), masks.to(device), lbls.to(device)
                outputs = model(input_ids=ids, attention_mask=masks, labels=lbls)
                val_loss += outputs.loss.item() * ids.size(0)
                logits = outputs.logits
                preds = (torch.sigmoid(logits) > threshold).int()
                all_true.append(lbls.int())
                all_pred.append(preds)
        val_loss /= len(dataloaders['val'].dataset)
        y_true = torch.cat(all_true, dim=0).cpu().numpy()
        y_pred = torch.cat(all_pred, dim=0).cpu().numpy()
        per_class_f1, val_f1 = compute_per_class_f1(y_true, y_pred)

        # Print per-class weighted F1 scores
        print(f"\n=== Validation Results (Epoch {epoch}):")
        for idx, label in enumerate(label_names):
            print(f"  - {label}: {per_class_f1[idx]:.4f}")
        print(f"  === Mean Weighted F1: {val_f1:.4f}")

        if val_loss < best_val_loss - eps or val_f1 > best_val_f1 + eps:
            best_val_loss = min(best_val_loss, val_loss)
            best_val_f1 = max(best_val_f1, val_f1)
            early_stop_counter = 0
        else:
            early_stop_counter += 1

    # Test
    model.eval()
    test_loss, all_true, all_pred = 0.0, [], []
    test_loop = tqdm(dataloaders['test'], desc="Testing", leave=False)
    with torch.no_grad():
        for ids, masks, lbls in test_loop:
            ids, masks, lbls = ids.to(device), masks.to(device), lbls.to(device)
            outputs = model(input_ids=ids, attention_mask=masks, labels=lbls)
            test_loss += outputs.loss.item() * ids.size(0)
            preds = (torch.sigmoid(outputs.logits) > threshold).int()
            all_true.append(lbls.int())
            all_pred.append(preds)
    test_loss /= len(dataloaders['test'].dataset)
    y_true = torch.cat(all_true, dim=0).cpu().numpy()
    y_pred = torch.cat(all_pred, dim=0).cpu().numpy()
    _, test_f1 = compute_per_class_f1(y_true, y_pred)

    return {
        "LR": learning_rate,
        "Epochs": num_epochs,
        "ValLoss": best_val_loss,
        "ValMeanF1": best_val_f1,
        "TestLoss": test_loss,
        "TestMeanF1": test_f1
    }

## Hyperparameter Search

In [ ]:
def run_hyperparameter_search( model_name, method='full', dataset_path='/content/finalDataset_0208.csv', output_dir='HyperParamSearchResults', learning_rates=[1e-5, 5e-5],
                              epochs_list=[3, 5], batch_size=16, seed=42 ):
    # making directory for saving the results
    os.makedirs(output_dir, exist_ok=True)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Prepare data ONCE
    dataloaders, label_names = prepare_data(
        model_name=model_name,
        dataset_path=dataset_path,
        batch_size=batch_size,
        seed=seed
    )

    all_results = []
    total_runs = len(learning_rates) * len(epochs_list)
    run_id = 0
    # looping through all the hyperparameters
    for lr in learning_rates:
        for epochs in epochs_list:
            run_id += 1
            print(f"\n🔍 Run {run_id}/{total_runs} | LR={lr}, Epochs={epochs}")
            result = train_model(
                model_name=model_name,
                method=method,
                learning_rate=lr,
                num_epochs=epochs,
                batch_size=batch_size,
                seed=seed,
                device=device,
                label_names=label_names,
                dataloaders=dataloaders
            )
            result["Method"] = method
            all_results.append(result)
    # saving all the results to the csv file, however we also print it
    df_results = pd.DataFrame(all_results)
    df_results.to_csv(f"{output_dir}/grid_{method}_{model_name.replace('/', '_')}.csv", index=False)

    best_row = df_results.loc[df_results["ValMeanF1"].idxmax()]
    print("\n==> Best Configuration Found:")
    print(best_row)


### Hyperparameter search for BERT in FULL fine tuning

Best hyperparameter which we found on this is:
- lr: 1e-5
- epoch:3

In [ ]:
run_hyperparameter_search(
    model_name="bert-base-uncased",
    method="full",
    learning_rates=[1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3],
    epochs_list=[3,4, 5],
    batch_size=8 # as in the original paper
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔍 Run 1/21 | LR=1e-06, Epochs=3
Trainable parameters in Bert case: 109,489,161 / 109,489,161 (100.00%)

=> Epoch 1/3



=== Validation Results (Epoch 1):
  - Price or Not: 0.7541
  - Direction Up: 0.4303
  - Direction Constant: 0.9386
  - Direction Down: 0.5114
  - PastPrice: 0.7119
  - FuturePrice: 0.9503
  - PastNews: 0.7652
  - FutureNews: 0.9882
  - Asset Comparision: 0.7017
  === Mean Weighted F1: 0.7502

=> Epoch 2/3



=== Validation Results (Epoch 2):
  - Price or Not: 0.7985
  - Direction Up: 0.6499
  - Direction Constant: 0.9386
  - Direction Down: 0.6773
  - PastPrice: 0.8488
  - FuturePrice: 0.9503
  - PastNews: 0.7673
  - FutureNews: 0.9882
  - Asset Comparision: 0.7099
  === Mean Weighted F1: 0.8143

=> Epoch 3/3



=== Validation Results (Epoch 3):
  - Price or Not: 0.9021
  - Direction Up: 0.8191
  - Direction Constant: 0.9386
  - Direction Down: 0.8523
  - PastPrice: 0.9123
  - FuturePrice: 0.9503
  - PastNews: 0.8908
  - FutureNews: 0.9882
  - Asset Comparision: 0.8951
  === Mean Weighted F1: 0.9054


...


=== Validation Results (Epoch 1):
  - Price or Not: 0.7541
  - Direction Up: 0.4177
  - Direction Constant: 0.9386
  - Direction Down: 0.5114
  - PastPrice: 0.7077
  - FuturePrice: 0.9503
  - PastNews: 0.7652
  - FutureNews: 0.9882
  - Asset Comparision: 0.7017
  === Mean Weighted F1: 0.7483

=> Epoch 2/5



=== Validation Results (Epoch 2):
  - Price or Not: 0.7541
  - Direction Up: 0.4177
  - Direction Constant: 0.9386
  - Direction Down: 0.5114
  - PastPrice: 0.7077
  - FuturePrice: 0.9503
  - PastNews: 0.7652
  - FutureNews: 0.9882
  - Asset Comparision: 0.7017
  === Mean Weighted F1: 0.7483

=> Epoch 3/5



=== Validation Results (Epoch 3):
  - Price or Not: 0.7541
  - Direction Up: 0.4177
  - Direction Constant: 0.9386
  - Direction Down: 0.5114
  - PastPrice: 0.7077
  - FuturePrice: 0.9503
  - PastNews: 0.7652
  - FutureNews: 0.9882
  - Asset Comparision: 0.7017
  === Mean Weighted F1: 0.7483

=> Epoch 4/5



=== Validation Results (Epoch 4):
  - Price or Not: 0.7541
  - Direction Up: 0.4177
  - Direction Constant: 0.9386
  - Direction Down: 0.5114
  - PastPrice: 0.7077
  - FuturePrice: 0.9503
  - PastNews: 0.7652
  - FutureNews: 0.9882
  - Asset Comparision: 0.7017
  === Mean Weighted F1: 0.7483
  ==> Early stopping triggered.



==> Best Configuration Found:
LR             0.00001
Epochs               3
ValLoss       0.098205
ValMeanF1     0.971015
TestLoss      0.096368
TestMeanF1    0.968794
Method            full
Name: 3, dtype: object


### Hyperparameter search for BERT in Lora fine tuning

Best hyperparameter which we found on this is:
- lr: 1e-3
- epoch: 3

In [ ]:
run_hyperparameter_search(
    model_name="bert-base-uncased",
    method="lora",
    learning_rates=[1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3],
    epochs_list=[3,4, 5],
    batch_size=16 # as in the original paper
)


🔍 Run 1/21 | LR=1e-06, Epochs=3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters in Lora case: 449,289 / 109,938,450 (0.41%)

=> Epoch 1/3



=== Validation Results (Epoch 1):
  - Price or Not: 0.7541
  - Direction Up: 0.3117
  - Direction Constant: 0.9386
  - Direction Down: 0.2034
  - PastPrice: 0.7077
  - FuturePrice: 0.9503
  - PastNews: 0.4158
  - FutureNews: 0.0001
  - Asset Comparision: 0.7017
  === Mean Weighted F1: 0.5537

=> Epoch 2/3



=== Validation Results (Epoch 2):
  - Price or Not: 0.7541
  - Direction Up: 0.3682
  - Direction Constant: 0.9386
  - Direction Down: 0.2197
  - PastPrice: 0.7077
  - FuturePrice: 0.9503
  - PastNews: 0.7588
  - FutureNews: 0.2641
  - Asset Comparision: 0.7017
  === Mean Weighted F1: 0.6293

=> Epoch 3/3


...


=== Validation Results (Epoch 2):
  - Price or Not: 0.9517
  - Direction Up: 0.9405
  - Direction Constant: 0.9617
  - Direction Down: 0.9396
  - PastPrice: 0.9455
  - FuturePrice: 0.9858
  - PastNews: 0.9487
  - FutureNews: 0.9877
  - Asset Comparision: 0.9921
  === Mean Weighted F1: 0.9615

=> Epoch 3/5



=== Validation Results (Epoch 3):
  - Price or Not: 0.9497
  - Direction Up: 0.9333
  - Direction Constant: 0.9739
  - Direction Down: 0.9474
  - PastPrice: 0.9359
  - FuturePrice: 0.9844
  - PastNews: 0.9459
  - FutureNews: 0.9882
  - Asset Comparision: 0.9965
  === Mean Weighted F1: 0.9617

=> Epoch 4/5



=== Validation Results (Epoch 4):
  - Price or Not: 0.9430
  - Direction Up: 0.9419
  - Direction Constant: 0.9742
  - Direction Down: 0.9481
  - PastPrice: 0.9448
  - FuturePrice: 0.9877
  - PastNews: 0.9440
  - FutureNews: 0.9882
  - Asset Comparision: 0.9956
  === Mean Weighted F1: 0.9630

=> Epoch 5/5



=== Validation Results (Epoch 5):
  - Price or Not: 0.7541
  - Direction Up: 0.4177
  - Direction Constant: 0.9386
  - Direction Down: 0.5114
  - PastPrice: 0.7077
  - FuturePrice: 0.9503
  - PastNews: 0.7652
  - FutureNews: 0.9882
  - Asset Comparision: 0.7017
  === Mean Weighted F1: 0.7483



==> Best Configuration Found:
LR               0.001
Epochs               3
ValLoss       0.108391
ValMeanF1     0.961674
TestLoss      0.108498
TestMeanF1    0.960967
Method            lora
Name: 18, dtype: object


### Hyperparameter search for BERT in Adapter fine tuning

Best hyperparameter which we found on this is:
- lr: 1e-3
- epoch: 6

In [ ]:
run_hyperparameter_search(
    model_name="bert-base-uncased",
    method="adapter",
    learning_rates=[1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3],
    epochs_list=[6,9,11],
    batch_size=16 # as in the original paper for Bert in Adapter
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...


=== Validation Results (Epoch 2):
  - Price or Not: 0.9425
  - Direction Up: 0.9375
  - Direction Constant: 0.9762
  - Direction Down: 0.9520
  - PastPrice: 0.9442
  - FuturePrice: 0.9921
  - PastNews: 0.9354
  - FutureNews: 0.9882
  - Asset Comparision: 0.9991
  === Mean Weighted F1: 0.9630

=> Epoch 3/11



=== Validation Results (Epoch 3):
  - Price or Not: 0.9397
  - Direction Up: 0.9135
  - Direction Constant: 0.9730
  - Direction Down: 0.9333
  - PastPrice: 0.9312
  - FuturePrice: 0.9868
  - PastNews: 0.9364
  - FutureNews: 0.9882
  - Asset Comparision: 0.9982
  === Mean Weighted F1: 0.9556

=> Epoch 4/11



=== Validation Results (Epoch 4):
  - Price or Not: 0.9580
  - Direction Up: 0.9396
  - Direction Constant: 0.9770
  - Direction Down: 0.9613
  - PastPrice: 0.9436
  - FuturePrice: 0.9753
  - PastNews: 0.9556
  - FutureNews: 0.9883
  - Asset Comparision: 1.0000
  === Mean Weighted F1: 0.9665

=> Epoch 5/11



=== Validation Results (Epoch 5):
  - Price or Not: 0.9460
  - Direction Up: 0.9319
  - Direction Constant: 0.9809
  - Direction Down: 0.9624
  - PastPrice: 0.9457
  - FuturePrice: 0.9902
  - PastNews: 0.9444
  - FutureNews: 0.9902
  - Asset Comparision: 1.0000
  === Mean Weighted F1: 0.9657
  ==> Early stopping triggered.



==> Best Configuration Found:
LR               0.001
Epochs               6
ValLoss       0.100429
ValMeanF1     0.963028
TestLoss      0.103347
TestMeanF1    0.963393
Method         adapter
Name: 18, dtype: object


## Training models with best hyperparameters

### Bert training

Full fine-tuning

In [ ]:
model_name = "bert-base-uncased"
method = "full"
lr = 1e-5
epochs = 3
batch_size = 8
seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_path = '/content/finalDataset_0208.csv'

dataloaders, label_names = prepare_data(
        model_name=model_name,
        dataset_path=dataset_path,
        batch_size=batch_size,
        seed=seed
    )

result = train_model(
                model_name=model_name,
                method=method,
                learning_rate=lr,
                num_epochs=epochs,
                batch_size=batch_size,
                seed=seed,
                device=device,
                label_names=label_names,
                dataloaders=dataloaders
            )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters in Bert case: 109,489,161 / 109,489,161 (100.00%)

=> Epoch 1/3



=== Validation Results (Epoch 1):
  - Price or Not: 0.9257
  - Direction Up: 0.9474
  - Direction Constant: 0.9386
  - Direction Down: 0.9536
  - PastPrice: 0.9164
  - FuturePrice: 0.9794
  - PastNews: 0.9235
  - FutureNews: 0.9882
  - Asset Comparision: 0.9930
  === Mean Weighted F1: 0.9518

=> Epoch 2/3



=== Validation Results (Epoch 2):
  - Price or Not: 0.9424
  - Direction Up: 0.9457
  - Direction Constant: 0.9521
  - Direction Down: 0.9563
  - PastPrice: 0.9271
  - FuturePrice: 0.9756
  - PastNews: 0.9399
  - FutureNews: 0.9882
  - Asset Comparision: 0.9991
  === Mean Weighted F1: 0.9585

=> Epoch 3/3



=== Validation Results (Epoch 3):
  - Price or Not: 0.9577
  - Direction Up: 0.9508
  - Direction Constant: 0.9791
  - Direction Down: 0.9666
  - PastPrice: 0.9532
  - FuturePrice: 0.9899
  - PastNews: 0.9545
  - FutureNews: 0.9882
  - Asset Comparision: 0.9991
  === Mean Weighted F1: 0.9710


In [ ]:
print("Full fine tuning result for News Headline Classification on BERT model",result["TestMeanF1"])

Full fine tuning result for News Headline Classification on BERT model 0.9687936119060078


Lora Fine tuning

In [ ]:
model_name = "bert-base-uncased"
method = "lora"
lr = 1e-3
epochs = 3
batch_size = 16
seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_path = '/content/finalDataset_0208.csv'

dataloaders, label_names = prepare_data(
        model_name=model_name,
        dataset_path=dataset_path,
        batch_size=batch_size,
        seed=seed
    )

result = train_model(
                model_name=model_name,
                method=method,
                learning_rate=lr,
                num_epochs=epochs,
                batch_size=batch_size,
                seed=seed,
                device=device,
                label_names=label_names,
                dataloaders=dataloaders
            )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters in Lora case: 449,289 / 109,938,450 (0.41%)

=> Epoch 1/3



=== Validation Results (Epoch 1):
  - Price or Not: 0.9455
  - Direction Up: 0.9211
  - Direction Constant: 0.9456
  - Direction Down: 0.9511
  - PastPrice: 0.9339
  - FuturePrice: 0.9818
  - PastNews: 0.9423
  - FutureNews: 0.9882
  - Asset Comparision: 0.9930
  === Mean Weighted F1: 0.9558

=> Epoch 2/3



=== Validation Results (Epoch 2):
  - Price or Not: 0.9517
  - Direction Up: 0.9405
  - Direction Constant: 0.9617
  - Direction Down: 0.9396
  - PastPrice: 0.9455
  - FuturePrice: 0.9858
  - PastNews: 0.9487
  - FutureNews: 0.9877
  - Asset Comparision: 0.9921
  === Mean Weighted F1: 0.9615

=> Epoch 3/3



=== Validation Results (Epoch 3):
  - Price or Not: 0.9497
  - Direction Up: 0.9333
  - Direction Constant: 0.9739
  - Direction Down: 0.9474
  - PastPrice: 0.9359
  - FuturePrice: 0.9844
  - PastNews: 0.9459
  - FutureNews: 0.9882
  - Asset Comparision: 0.9965
  === Mean Weighted F1: 0.9617


In [ ]:
print("Lora fine tuning result for News Headline Classification on BERT model",result["TestMeanF1"])

Lora fine tuning result for News Headline Classification on BERT model 0.9609673319412221


Adapter Fine tuning

In [ ]:
model_name = "bert-base-uncased"
method = "adapter"
lr = 1e-3
epochs = 6
batch_size = 16
seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_path = '/content/finalDataset_0208.csv'

dataloaders, label_names = prepare_data(
        model_name=model_name,
        dataset_path=dataset_path,
        batch_size=batch_size,
        seed=seed
    )

result = train_model(
                model_name=model_name,
                method=method,
                learning_rate=lr,
                num_epochs=epochs,
                batch_size=batch_size,
                seed=seed,
                device=device,
                label_names=label_names,
                dataloaders=dataloaders
            )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters in Adapter case: 1,795,977 / 111,278,217 (1.61%)

=> Epoch 1/6



=== Validation Results (Epoch 1):
  - Price or Not: 0.9079
  - Direction Up: 0.9196
  - Direction Constant: 0.9559
  - Direction Down: 0.9546
  - PastPrice: 0.9156
  - FuturePrice: 0.9885
  - PastNews: 0.9001
  - FutureNews: 0.9882
  - Asset Comparision: 0.9956
  === Mean Weighted F1: 0.9473

=> Epoch 2/6



=== Validation Results (Epoch 2):
  - Price or Not: 0.9425
  - Direction Up: 0.9375
  - Direction Constant: 0.9762
  - Direction Down: 0.9520
  - PastPrice: 0.9442
  - FuturePrice: 0.9921
  - PastNews: 0.9354
  - FutureNews: 0.9882
  - Asset Comparision: 0.9991
  === Mean Weighted F1: 0.9630

=> Epoch 3/6



=== Validation Results (Epoch 3):
  - Price or Not: 0.9397
  - Direction Up: 0.9135
  - Direction Constant: 0.9730
  - Direction Down: 0.9333
  - PastPrice: 0.9312
  - FuturePrice: 0.9868
  - PastNews: 0.9364
  - FutureNews: 0.9882
  - Asset Comparision: 0.9982
  === Mean Weighted F1: 0.9556

=> Epoch 4/6



=== Validation Results (Epoch 4):
  - Price or Not: 0.9580
  - Direction Up: 0.9396
  - Direction Constant: 0.9770
  - Direction Down: 0.9613
  - PastPrice: 0.9436
  - FuturePrice: 0.9753
  - PastNews: 0.9556
  - FutureNews: 0.9883
  - Asset Comparision: 1.0000
  === Mean Weighted F1: 0.9665

=> Epoch 5/6



=== Validation Results (Epoch 5):
  - Price or Not: 0.9460
  - Direction Up: 0.9319
  - Direction Constant: 0.9809
  - Direction Down: 0.9624
  - PastPrice: 0.9457
  - FuturePrice: 0.9902
  - PastNews: 0.9444
  - FutureNews: 0.9902
  - Asset Comparision: 1.0000
  === Mean Weighted F1: 0.9657
  ==> Early stopping triggered.


In [ ]:
print("Adapter fine tuning result for News Headline Classification on BERT model",result["TestMeanF1"])

Adapter fine tuning result for News Headline Classification on BERT model 0.9633931512407339


### FinBert training

Full fine tuning

In [ ]:
model_name = "yiyanghkust/finbert-pretrain"
method = "full"
lr = 1e-5
epochs = 3
batch_size = 8
seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_path = '/content/finalDataset_0208.csv'

dataloaders, label_names = prepare_data(
        model_name=model_name,
        dataset_path=dataset_path,
        batch_size=batch_size,
        seed=seed
    )

result = train_model(
                model_name=model_name,
                method=method,
                learning_rate=lr,
                num_epochs=epochs,
                batch_size=batch_size,
                seed=seed,
                device=device,
                label_names=label_names,
                dataloaders=dataloaders
            )

config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Training:   0%|          | 4/999 [00:00<01:39, 10.04it/s, loss=0.66]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]


=== Validation Results (Epoch 1):
  - Price or Not: 0.9372
  - Direction Up: 0.9401
  - Direction Constant: 0.9704
  - Direction Down: 0.9580
  - PastPrice: 0.9195
  - FuturePrice: 0.9769
  - PastNews: 0.9254
  - FutureNews: 0.9882
  - Asset Comparision: 0.9938
  === Mean Weighted F1: 0.9566

=> Epoch 2/3



=== Validation Results (Epoch 2):
  - Price or Not: 0.9510
  - Direction Up: 0.9553
  - Direction Constant: 0.9786
  - Direction Down: 0.9615
  - PastPrice: 0.9432
  - FuturePrice: 0.9870
  - PastNews: 0.9452
  - FutureNews: 0.9882
  - Asset Comparision: 0.9974
  === Mean Weighted F1: 0.9675

=> Epoch 3/3



=== Validation Results (Epoch 3):
  - Price or Not: 0.9544
  - Direction Up: 0.9499
  - Direction Constant: 0.9772
  - Direction Down: 0.9587
  - PastPrice: 0.9508
  - FuturePrice: 0.9876
  - PastNews: 0.9483
  - FutureNews: 0.9895
  - Asset Comparision: 0.9991
  === Mean Weighted F1: 0.9684


In [ ]:
print("Full fine tuning result for News Headline Classification on BERT model",result["TestMeanF1"])

Full fine tuning result for News Headline Classification on BERT model 0.9674159313667696


Lora fine tuning

In [ ]:
model_name = "yiyanghkust/finbert-pretrain"
method = "lora"
lr = 1e-3
epochs = 3
batch_size = 16
seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_path = '/content/finalDataset_0208.csv'

dataloaders, label_names = prepare_data(
        model_name=model_name,
        dataset_path=dataset_path,
        batch_size=batch_size,
        seed=seed
    )

result = train_model(
                model_name=model_name,
                method=method,
                learning_rate=lr,
                num_epochs=epochs,
                batch_size=batch_size,
                seed=seed,
                device=device,
                label_names=label_names,
                dataloaders=dataloaders
            )

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters in Lora case: 449,289 / 110,208,018 (0.41%)

=> Epoch 1/3



=== Validation Results (Epoch 1):
  - Price or Not: 0.9366
  - Direction Up: 0.9162
  - Direction Constant: 0.9706
  - Direction Down: 0.9553
  - PastPrice: 0.9219
  - FuturePrice: 0.9836
  - PastNews: 0.9294
  - FutureNews: 0.9882
  - Asset Comparision: 0.9956
  === Mean Weighted F1: 0.9553

=> Epoch 2/3



=== Validation Results (Epoch 2):
  - Price or Not: 0.9420
  - Direction Up: 0.9368
  - Direction Constant: 0.9777
  - Direction Down: 0.9554
  - PastPrice: 0.9420
  - FuturePrice: 0.9847
  - PastNews: 0.9422
  - FutureNews: 0.9900
  - Asset Comparision: 0.9938
  === Mean Weighted F1: 0.9627

=> Epoch 3/3



=== Validation Results (Epoch 3):
  - Price or Not: 0.9347
  - Direction Up: 0.9307
  - Direction Constant: 0.9738
  - Direction Down: 0.9388
  - PastPrice: 0.9301
  - FuturePrice: 0.9849
  - PastNews: 0.9289
  - FutureNews: 0.9877
  - Asset Comparision: 0.9912
  === Mean Weighted F1: 0.9556


In [ ]:
print("Lora fine tuning result for News Headline Classification on BERT model",result["TestMeanF1"])

Lora fine tuning result for News Headline Classification on BERT model 0.958404170123917


Adapter fine tuning

In [ ]:
model_name = "yiyanghkust/finbert-pretrain"
method = "adapter"
lr = 1e-3
epochs = 6
batch_size = 16
seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_path = '/content/finalDataset_0208.csv'

dataloaders, label_names = prepare_data(
        model_name=model_name,
        dataset_path=dataset_path,
        batch_size=batch_size,
        seed=seed
    )

result = train_model(
                model_name=model_name,
                method=method,
                learning_rate=lr,
                num_epochs=epochs,
                batch_size=batch_size,
                seed=seed,
                device=device,
                label_names=label_names,
                dataloaders=dataloaders
            )

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters in Adapter case: 1,795,977 / 111,547,785 (1.61%)

=> Epoch 1/6



=== Validation Results (Epoch 1):
  - Price or Not: 0.8883
  - Direction Up: 0.9357
  - Direction Constant: 0.9733
  - Direction Down: 0.9300
  - PastPrice: 0.8983
  - FuturePrice: 0.9879
  - PastNews: 0.8835
  - FutureNews: 0.9877
  - Asset Comparision: 0.9878
  === Mean Weighted F1: 0.9414

=> Epoch 2/6



=== Validation Results (Epoch 2):
  - Price or Not: 0.9393
  - Direction Up: 0.9455
  - Direction Constant: 0.9787
  - Direction Down: 0.9545
  - PastPrice: 0.9428
  - FuturePrice: 0.9892
  - PastNews: 0.9349
  - FutureNews: 0.9894
  - Asset Comparision: 0.9982
  === Mean Weighted F1: 0.9636

=> Epoch 3/6



=== Validation Results (Epoch 3):
  - Price or Not: 0.9369
  - Direction Up: 0.9457
  - Direction Constant: 0.9802
  - Direction Down: 0.9266
  - PastPrice: 0.9309
  - FuturePrice: 0.9896
  - PastNews: 0.9333
  - FutureNews: 0.9882
  - Asset Comparision: 1.0000
  === Mean Weighted F1: 0.9590

=> Epoch 4/6



=== Validation Results (Epoch 4):
  - Price or Not: 0.9469
  - Direction Up: 0.9396
  - Direction Constant: 0.9779
  - Direction Down: 0.9560
  - PastPrice: 0.9436
  - FuturePrice: 0.9907
  - PastNews: 0.9399
  - FutureNews: 0.9895
  - Asset Comparision: 0.9982
  === Mean Weighted F1: 0.9647

=> Epoch 5/6



=== Validation Results (Epoch 5):
  - Price or Not: 0.9521
  - Direction Up: 0.9509
  - Direction Constant: 0.9835
  - Direction Down: 0.9477
  - PastPrice: 0.9470
  - FuturePrice: 0.9921
  - PastNews: 0.9438
  - FutureNews: 0.9864
  - Asset Comparision: 0.9982
  === Mean Weighted F1: 0.9669
  ==> Early stopping triggered.


In [ ]:
print("Adapter fine tuning result for News Headline Classification on BERT model",result["TestMeanF1"])

Adapter fine tuning result for News Headline Classification on BERT model 0.9674231836641243


### FlangBert training

Full fine tuning

In [ ]:
model_name = "SALT-NLP/FLANG-BERT"
method = "full"
lr = 1e-5
epochs = 3
batch_size = 8
seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_path = '/content/finalDataset_0208.csv'

dataloaders, label_names = prepare_data(
        model_name=model_name,
        dataset_path=dataset_path,
        batch_size=batch_size,
        seed=seed
    )

result = train_model(
                model_name=model_name,
                method=method,
                learning_rate=lr,
                num_epochs=epochs,
                batch_size=batch_size,
                seed=seed,
                device=device,
                label_names=label_names,
                dataloaders=dataloaders
            )

tokenizer_config.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at SALT-NLP/FLANG-BERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Training:   1%|          | 5/999 [00:00<01:37, 10.15it/s, loss=0.615]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]


=== Validation Results (Epoch 1):
  - Price or Not: 0.9272
  - Direction Up: 0.9422
  - Direction Constant: 0.9704
  - Direction Down: 0.9544
  - PastPrice: 0.9182
  - FuturePrice: 0.9794
  - PastNews: 0.9207
  - FutureNews: 0.9882
  - Asset Comparision: 0.9921
  === Mean Weighted F1: 0.9548

=> Epoch 2/3



=== Validation Results (Epoch 2):
  - Price or Not: 0.9403
  - Direction Up: 0.9483
  - Direction Constant: 0.9815
  - Direction Down: 0.9564
  - PastPrice: 0.9324
  - FuturePrice: 0.9864
  - PastNews: 0.9369
  - FutureNews: 0.9882
  - Asset Comparision: 0.9974
  === Mean Weighted F1: 0.9631

=> Epoch 3/3



=== Validation Results (Epoch 3):
  - Price or Not: 0.9472
  - Direction Up: 0.9527
  - Direction Constant: 0.9811
  - Direction Down: 0.9607
  - PastPrice: 0.9460
  - FuturePrice: 0.9900
  - PastNews: 0.9400
  - FutureNews: 0.9882
  - Asset Comparision: 0.9991
  === Mean Weighted F1: 0.9672


In [ ]:
print("Full fine tuning result for News Headline Classification on BERT model",result["TestMeanF1"])

Full fine tuning result for News Headline Classification on BERT model 0.96784471634007


Lora fine tuning

In [ ]:
model_name = "SALT-NLP/FLANG-BERT"
method = "lora"
lr = 1e-3
epochs = 3
batch_size = 16
seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_path = '/content/finalDataset_0208.csv'

dataloaders, label_names = prepare_data(
        model_name=model_name,
        dataset_path=dataset_path,
        batch_size=batch_size,
        seed=seed
    )

result = train_model(
                model_name=model_name,
                method=method,
                learning_rate=lr,
                num_epochs=epochs,
                batch_size=batch_size,
                seed=seed,
                device=device,
                label_names=label_names,
                dataloaders=dataloaders
            )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at SALT-NLP/FLANG-BERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters in Lora case: 449,289 / 109,938,450 (0.41%)

=> Epoch 1/3



=== Validation Results (Epoch 1):
  - Price or Not: 0.9467
  - Direction Up: 0.9413
  - Direction Constant: 0.9727
  - Direction Down: 0.9467
  - PastPrice: 0.9321
  - FuturePrice: 0.9844
  - PastNews: 0.9381
  - FutureNews: 0.9877
  - Asset Comparision: 0.9974
  === Mean Weighted F1: 0.9608

=> Epoch 2/3



=== Validation Results (Epoch 2):
  - Price or Not: 0.9468
  - Direction Up: 0.9369
  - Direction Constant: 0.9793
  - Direction Down: 0.9556
  - PastPrice: 0.9418
  - FuturePrice: 0.9844
  - PastNews: 0.9409
  - FutureNews: 0.9870
  - Asset Comparision: 0.9974
  === Mean Weighted F1: 0.9633

=> Epoch 3/3



=== Validation Results (Epoch 3):
  - Price or Not: 0.9531
  - Direction Up: 0.9378
  - Direction Constant: 0.9785
  - Direction Down: 0.9531
  - PastPrice: 0.9389
  - FuturePrice: 0.9880
  - PastNews: 0.9454
  - FutureNews: 0.9882
  - Asset Comparision: 0.9982
  === Mean Weighted F1: 0.9646


In [ ]:
print("Lora fine tuning result for News Headline Classification on BERT model",result["TestMeanF1"])

Lora fine tuning result for News Headline Classification on BERT model 0.9647934153159602


Adapter Fine tuning

In [ ]:
model_name = "SALT-NLP/FLANG-BERT"
method = "adapter"
lr = 1e-3
epochs = 6
batch_size = 16
seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_path = '/content/finalDataset_0208.csv'

dataloaders, label_names = prepare_data(
        model_name=model_name,
        dataset_path=dataset_path,
        batch_size=batch_size,
        seed=seed
    )

result = train_model(
                model_name=model_name,
                method=method,
                learning_rate=lr,
                num_epochs=epochs,
                batch_size=batch_size,
                seed=seed,
                device=device,
                label_names=label_names,
                dataloaders=dataloaders
            )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at SALT-NLP/FLANG-BERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters in Adapter case: 1,795,977 / 111,278,217 (1.61%)

=> Epoch 1/6



=== Validation Results (Epoch 1):
  - Price or Not: 0.9489
  - Direction Up: 0.9244
  - Direction Constant: 0.9627
  - Direction Down: 0.9327
  - PastPrice: 0.9459
  - FuturePrice: 0.9900
  - PastNews: 0.9498
  - FutureNews: 0.9882
  - Asset Comparision: 0.9858
  === Mean Weighted F1: 0.9587

=> Epoch 2/6



=== Validation Results (Epoch 2):
  - Price or Not: 0.9270
  - Direction Up: 0.9275
  - Direction Constant: 0.9802
  - Direction Down: 0.9257
  - PastPrice: 0.9201
  - FuturePrice: 0.9858
  - PastNews: 0.9190
  - FutureNews: 0.9894
  - Asset Comparision: 0.9982
  === Mean Weighted F1: 0.9526

=> Epoch 3/6



=== Validation Results (Epoch 3):
  - Price or Not: 0.9536
  - Direction Up: 0.9491
  - Direction Constant: 0.9699
  - Direction Down: 0.9588
  - PastPrice: 0.9483
  - FuturePrice: 0.9844
  - PastNews: 0.9513
  - FutureNews: 0.9882
  - Asset Comparision: 0.9991
  === Mean Weighted F1: 0.9670

=> Epoch 4/6



=== Validation Results (Epoch 4):
  - Price or Not: 0.9232
  - Direction Up: 0.9371
  - Direction Constant: 0.9783
  - Direction Down: 0.9553
  - PastPrice: 0.9239
  - FuturePrice: 0.9909
  - PastNews: 0.9227
  - FutureNews: 0.9882
  - Asset Comparision: 0.9982
  === Mean Weighted F1: 0.9575

=> Epoch 5/6



=== Validation Results (Epoch 5):
  - Price or Not: 0.9495
  - Direction Up: 0.9312
  - Direction Constant: 0.9792
  - Direction Down: 0.9544
  - PastPrice: 0.9426
  - FuturePrice: 0.9899
  - PastNews: 0.9490
  - FutureNews: 0.9882
  - Asset Comparision: 0.9965
  === Mean Weighted F1: 0.9645

=> Epoch 6/6



=== Validation Results (Epoch 6):
  - Price or Not: 0.9501
  - Direction Up: 0.9492
  - Direction Constant: 0.9842
  - Direction Down: 0.9530
  - PastPrice: 0.9472
  - FuturePrice: 0.9893
  - PastNews: 0.9472
  - FutureNews: 0.9882
  - Asset Comparision: 1.0000
  === Mean Weighted F1: 0.9676


In [ ]:
print("Adapter fine tuning result for News Headline Classification on BERT model",result["TestMeanF1"])

Adapter fine tuning result for News Headline Classification on BERT model 0.968865551473793


# Named Entity Recognition

## Data loading and preprocessing

Dataset is in my google drive, so links is dependent on my drive. If you need to run it by yourself you need to get the datasets from: https://huggingface.co/datasets/tner/fin

- We tried to find it on the paper, however there is no data on it in the paper. However, we found the same dataset on huggingface by looking the references in the paper (like they referenced another paper and we looked at this referenced paper)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    get_linear_schedule_with_warmup
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import f1_score
from tqdm import tqdm
import adapters
from adapters import AdapterConfig

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def load_data(tokenizer, train=True, label_all_tokens=True):
    """
    Load and preprocess NER data

    Args:
        tokenizer: HuggingFace tokenizer
        train: load training data or testing data
        label_all_tokens: Whether to label all tokens or just the first token of a word

    Returns:
        dataset: TensorDataset containing input_ids, attention_masks, and labels
        num_labels: Number of unique labels
        label_to_id: Dictionary mapping labels to IDs
    """

    # file path will change depending on your drive, make sure to put correct path
    if train:
        file_name = '/content/drive/MyDrive/application_nlp cv/advanced_nlp/FIN5.txt'
    else:
        file_name = '/content/drive/MyDrive/application_nlp cv/advanced_nlp/FIN3.txt'

    with open(file_name, encoding="utf-8") as f:
        lines = [line.rstrip() for line in f]

    sentences_tags = []
    sentence_tags = []
    sentences = []
    sentence = []
    i = 0
    max_length = 0
    unique_tags = set()

    # getting sentences and corresponding tokens for each word
    while i < len(lines):
        if not lines[i] and sentence and sentence_tags:
            tokens = tokenizer(sentence, is_split_into_words=True)
            if len(tokens['input_ids']) > 512:
                sentence = []
                sentence_tags = []
                continue
            max_length = max(max_length, len(tokens['input_ids']))
            sentences.append(sentence)
            sentences_tags.append(sentence_tags)
            unique_tags = unique_tags | set(sentence_tags)
            sentence = []
            sentence_tags = []
        elif lines[i] and "DOCSTART" not in lines[i]:
            word, _, _, tag = lines[i].split(" ")
            sentence.append(word)
            sentence_tags.append(tag)
        i += 1

    label_list = list(unique_tags)
    label_list.sort()
    label_to_id = {l: i for i, l in enumerate(label_list)}
    print(label_to_id)
    num_labels = len(label_list)

    # giving it to tokenizer
    tokenized_inputs = tokenizer(sentences, max_length=max_length, padding='max_length', is_split_into_words=True, return_tensors='pt')
    input_ids = tokenized_inputs['input_ids']
    attention_masks = tokenized_inputs['attention_mask']

    # getting labels
    labels = []
    for i, label in enumerate(sentences_tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            else:
                label_ids.append(label_to_id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    labels = torch.LongTensor(labels)

    return TensorDataset(input_ids, attention_masks, labels), num_labels, label_to_id

In [ ]:
def prepare_datasets(model_name, label_all_tokens=True):
    """
    Prepare datasets for training, validation, and testing

    Args:
        model_name: Name of the pretrained model (as we are going to use 3 Bert models)
        label_all_tokens: Whether to label all tokens or just the first token of a word

    Returns:
        train_dataset: Training dataset
        val_dataset: Validation dataset
        test_dataset: Test dataset
        num_labels: Number of unique labels
        label_to_id: Dictionary mapping labels to IDs
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, do_basic_tokenize=True)

    train_dataset, num_labels, label_to_id = load_data(tokenizer, train=True, label_all_tokens=label_all_tokens)
    test_dataset, _, _ = load_data(tokenizer, train=False, label_all_tokens=label_all_tokens)

    # splitting val and train data
    val_length = int(len(train_dataset) * 0.2)
    train_length = len(train_dataset) - val_length

    # Create loss weights (as we will focus on 4 classes except 'O' label)
    weights = torch.ones(num_labels).to(device) * 1 / (num_labels - 1)
    weights[weights.size(0) - 1] = 0
    print(weights)
    print("length_dataset", train_length, val_length, len(test_dataset))

    return train_dataset, num_labels, label_to_id, test_dataset, train_length, val_length, weights

In [ ]:
def get_model(model_name, num_labels, fine_tuning_method="full"):
    """
    Get model based on fine-tuning method

    Args:
        model_name: Name of the pretrained model
        num_labels: Number of unique labels
        fine_tuning_method: One of "full", "lora", or "adapter"

    Returns:
        model: Model with specified fine-tuning configuration
    """
    model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

    if fine_tuning_method == "lora":
        # LoRA configuration
        lora_config = LoraConfig(
            task_type=TaskType.TOKEN_CLS,
            r=16,
            lora_alpha=16,
            lora_dropout=0.05,
            target_modules=["query", "key", "value"],
            bias="none"
        )
        model = get_peft_model(model, lora_config)
    elif fine_tuning_method == "adapter":
        # Adapter configuration
        adapter_config = AdapterConfig.load("houlsby")
        adapters.init(model)
        adapter_name = "ner_adapter"
        model.add_adapter(adapter_name, config=adapter_config)
        model.train_adapter(adapter_name)
        model.set_active_adapters(adapter_name)

    model.to(device)

    # Print trainable parameters (to check are we really applying parameter efficient fine tuning methods)
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Trainable parameters: {trainable_params:,} / {total_params:,}",
          f"({100 * trainable_params / total_params:.2f}%)")

    return model

In [ ]:
def train_and_evaluate(
    model_name,
    fine_tuning_method="full",
    batch_sizes=[16],
    learning_rates=[1e-4],
    num_epochs=5,
    early_stopping=3,
    label_all_tokens=True,
    seed=42
):
    """
    Train and evaluate model with specified hyperparameters

    Args:
        model_name: Name of the pretrained model
        fine_tuning_method: One of "full", "lora", or "adapter"
        batch_sizes: List of batch sizes to try (as we are limited with computation power, we decided to fix batch size which was shown on the paper)
        learning_rates: List of learning rates to try
        num_epochs: Maximum number of epochs to train
        early_stopping: Number of epochs with no improvement before stopping
        label_all_tokens: Whether to label all tokens or just the first token of a word
        seed: Random seed

    Returns:
        results: List of results for each hyperparameter configuration
    """
    # Prepare datasets
    train_dataset, num_labels, label_to_id, test_dataset, train_length, val_length, weights = prepare_datasets(
        model_name, label_all_tokens
    )

    # Set loss function
    criterion = torch.nn.CrossEntropyLoss(weight=weights)

    results = []
    eps = 1e-2
    count = 0

    for j, lr in enumerate(learning_rates):
        for k, bs in enumerate(batch_sizes):
            count += 1
            print(f'\n== Experiment {count} of {len(batch_sizes) * len(learning_rates)}:')
            print(f'=> Learning Rate: {lr}, Batch Size: {bs}')

            # Set random seeds
            torch.manual_seed(seed)
            np.random.seed(seed)

            # Split datasets
            train, val = random_split(dataset=train_dataset, lengths=[train_length, val_length])
            dataloaders_dict = {
                'train': DataLoader(train, batch_size=bs, shuffle=True),
                'val': DataLoader(val, batch_size=bs, shuffle=False),
                'test': DataLoader(test_dataset, batch_size=bs, shuffle=False)
            }

            # Get model
            model = get_model(model_name, num_labels, fine_tuning_method)

            # Setup optimizer
            optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

            # Add learning rate scheduler
            total_steps = len(dataloaders_dict['train']) * num_epochs
            scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=int(0.1 * total_steps),  # 10% warmup
                num_training_steps=total_steps
            )

            early_stopping_count = 0
            best_accuracy = float('-inf')
            best_f1 = float('-inf')
            best_ce = float('inf')

            # For numerical stability tracking (in some cases model is giving nan logits, so i will decided to skip this batches for training)
            prev_loss = None

            for epoch in range(num_epochs):
                print(f'\n=== Epoch {epoch + 1}')
                if early_stopping_count >= early_stopping:
                    break

                for phase in ['train', 'val']:
                    print(f'-- {phase.upper()} Phase')
                    if phase == 'train':
                        model.train()
                        early_stopping_count += 1
                    else:
                        model.eval()

                    curr_total = 0
                    curr_correct = 0
                    curr_accuracy = 0
                    curr_ce = 0
                    actual = np.array([])
                    pred = np.array([])

                    # Track running loss for monitoring
                    running_loss = 0.0
                    num_batches = 0

                    for input_ids, attention_masks, labels in tqdm(dataloaders_dict[phase], desc=f"{phase.capitalize()} Loop"):
                        input_ids = input_ids.to(device)
                        attention_masks = attention_masks.to(device)
                        labels = labels.to(device)

                        optimizer.zero_grad()

                        with torch.set_grad_enabled(phase == 'train'):
                            outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)

                            # Handle potential NaN values in logits
                            if torch.isnan(outputs.logits).any():
                                print(f"Warning: NaN detected in logits - skipping batch")
                                continue

                            active_loss = attention_masks.view(-1) == 1
                            logits = outputs.logits
                            active_logits = logits.view(-1, num_labels)
                            active_labels = torch.where(
                                active_loss, labels.view(-1), torch.tensor(criterion.ignore_index).type_as(labels)
                            )

                            # Check if we have valid elements for loss calculation
                            if torch.sum(active_loss) == 0:
                                print("Warning: No active elements for loss calculation")
                                continue

                            # Loss calculation with safety checks
                            try:
                                loss = criterion(active_logits, active_labels)

                                # Skip batch if loss is NaN
                                if torch.isnan(loss).any():
                                    print(f"Warning: NaN loss detected - skipping batch")
                                    continue

                                # Track running loss
                                running_loss += loss.item()
                                num_batches += 1

                                if phase == 'train':
                                    loss.backward()

                                    # Gradient clipping to prevent explosion
                                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                                    optimizer.step()
                                    scheduler.step()
                                else:
                                    curr_pred = outputs.logits.argmax(dim=-1).detach().cpu().clone().numpy()
                                    curr_actual = labels.detach().cpu().clone().numpy()

                                    # Safe extraction of valid predictions and labels
                                    batch_true_predictions = []
                                    batch_true_labels = []

                                    for pred_seq, gold_seq in zip(curr_pred, curr_actual):
                                        valid_pairs = [(p, l) for p, l in zip(pred_seq, gold_seq)
                                                      if l != -100 and l != label_to_id['O']]
                                        if valid_pairs:
                                            batch_preds, batch_labels = zip(*valid_pairs)
                                            batch_true_predictions.extend(batch_preds)
                                            batch_true_labels.extend(batch_labels)

                                    if batch_true_predictions:
                                        curr_correct += np.sum(np.array(batch_true_predictions) == np.array(batch_true_labels))
                                        curr_total += len(batch_true_predictions)
                                        curr_ce += loss.item() * input_ids.size(0)

                                        # Safely extend our arrays
                                        if len(batch_true_labels) > 0:
                                            if len(actual) == 0:
                                                actual = np.array(batch_true_labels)
                                                pred = np.array(batch_true_predictions)
                                            else:
                                                actual = np.concatenate([actual, np.array(batch_true_labels)], axis=0)
                                                pred = np.concatenate([pred, np.array(batch_true_predictions)], axis=0)
                            except Exception as e:
                                print(f"Error in loss calculation: {e}")
                                continue

                    # Report average loss for this epoch
                    if num_batches > 0:
                        epoch_loss = running_loss / num_batches
                        print(f"{phase} Loss: {epoch_loss:.4f}")

                        # Check for dramatic loss increases that might indicate instability
                        if phase == 'train' and prev_loss is not None:
                            if epoch_loss > prev_loss * 2:  # Loss more than doubled
                                print(f"Warning: Loss increased dramatically from {prev_loss:.4f} to {epoch_loss:.4f}")

                        if phase == 'train':
                            prev_loss = epoch_loss

                    if phase == 'val':
                        if curr_total > 0:  # Make sure we have valid samples
                            curr_accuracy = curr_correct / curr_total
                            curr_f1 = f1_score(actual, pred, average='weighted') if len(actual) > 0 else 0.0
                            curr_ce = curr_ce / len(val) if len(val) > 0 else float('inf')

                            if curr_ce <= best_ce + eps:
                                best_ce = curr_ce
                                early_stopping_count = 0
                            if curr_accuracy >= best_accuracy + eps:
                                best_accuracy = curr_accuracy
                                early_stopping_count = 0
                            if curr_f1 >= best_f1 + eps:
                                best_f1 = curr_f1
                                early_stopping_count = 0

                            print("== Val Cross Entropy: ", curr_ce)
                            print("== Val Accuracy: ", curr_accuracy)
                            print("== Val F1: ", curr_f1)
                            print("== Early Stopping Count: ", early_stopping_count)

            # Test phase
            test_accuracy = 0
            test_total = 0
            test_correct = 0
            actual = np.array([])
            pred = np.array([])

            model.eval()  # Ensure model is in evaluation mode

            for input_ids, attention_masks, labels in tqdm(dataloaders_dict['test'], desc="Testing"):
                input_ids = input_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.to(device)

                with torch.no_grad():
                    outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)

                    curr_pred = outputs.logits.argmax(dim=-1).detach().cpu().clone().numpy()
                    curr_actual = labels.detach().cpu().clone().numpy()
                    true_predictions = np.concatenate([
                        [p for (p, l) in zip(pred, gold_label) if l != -100 and l != label_to_id['O']]
                        for pred, gold_label in zip(curr_pred, curr_actual)
                    ])
                    true_labels = np.concatenate([
                        [l for (p, l) in zip(pred, gold_label) if l != -100 and l != label_to_id['O']]
                        for pred, gold_label in zip(curr_pred, curr_actual)
                    ])

                    test_total += len(true_predictions)
                    test_correct += np.sum(true_predictions == true_labels)
                    actual = np.concatenate([actual, true_labels], axis=0)
                    pred = np.concatenate([pred, true_predictions], axis=0)

            test_accuracy = test_correct / test_total
            test_f1 = f1_score(actual, pred, average='weighted') if len(actual) > 0 else 0.0

            print(f"\n Test Results — LR: {lr}, BS: {bs}")
            print(f" Test Accuracy: {test_accuracy:.4f}")
            print(f" Test F1 Score: {test_f1:.4f}")

            results.append([seed, lr, bs, best_ce, best_accuracy, best_f1, test_accuracy, test_f1])

    return results

In [ ]:
def hyperparameter_search(
    model_name,
    fine_tuning_methods=["full", "lora", "adapter"],
    learning_rates=[1e-5, 1e-4, 1e-3],
    batch_sizes=[16, 32],
    epochs=[3, 5, 7],
    label_all_tokens=True,
    seed=42
):
    """
    Perform hyperparameter search across different fine-tuning methods

    Args:
        model_name: Name of the pretrained model
        fine_tuning_methods: List of fine-tuning methods to try
        learning_rates: List of learning rates to try
        batch_sizes: List of batch sizes to try
        epochs: List of max epochs to try
        label_all_tokens: Whether to label all tokens or just the first token of a word
        seed: Random seed

    Returns:
        all_results: Dictionary mapping fine-tuning methods to their results
    """
    all_results = {}

    for method in fine_tuning_methods:
        print(f"\n{'=' * 50}")
        print(f"Running experiments for {method} fine-tuning on {model_name}")
        print(f"{'=' * 50}")

        method_results = []

        for num_epochs in epochs:
            results = train_and_evaluate(
                model_name=model_name,
                fine_tuning_method=method,
                batch_sizes=batch_sizes,
                learning_rates=learning_rates,
                num_epochs=num_epochs,
                label_all_tokens=label_all_tokens,
                seed=seed
            )

            for result in results:
                # Add fine-tuning method and epochs to results
                method_results.append([method, num_epochs] + result)

        all_results[method] = method_results

    return all_results

In [ ]:
# this function goes through methods as loop, however we will run only one method in each cell. Maybe in the future for getting the best method among all
# other this function will be helpful. So, i decided to left it.
def display_results(all_results):
    """
    Display results from hyperparameter search

    Args:
        all_results: Dictionary mapping fine-tuning methods to their results
    """
    print("\n" + "=" * 80)
    print("SUMMARY OF RESULTS")
    print("=" * 80)

    for method, results in all_results.items():
        print(f"\n{method.upper()} FINE-TUNING RESULTS:")
        print("-" * 60)
        print(f"{'Method':<8} {'Epochs':<6} {'LR':<8} {'BS':<4} {'Val CE':<8} {'Val Acc':<8} {'Val F1':<8} {'Test Acc':<8} {'Test F1':<8}")
        print("-" * 60)

        # Sort by test F1 score (descending) (however we choose best one depending on val F1 as it is hyperparameter search)
        sorted_results = sorted(results, key=lambda x: x[-1], reverse=True)

        for result in sorted_results:
            method, epochs, seed, lr, bs, val_ce, val_acc, val_f1, test_acc, test_f1 = result
            print(f"{method:<8} {epochs:<6d} {lr:<8.6f} {bs:<4d} {val_ce:<8.4f} {val_acc:<8.4f} {val_f1:<8.4f} {test_acc:<8.4f} {test_f1:<8.4f}")

    # Find best overall configuration
    best_method = None
    best_config = None
    best_f1 = -1

    for method, results in all_results.items():
        for result in results:
            if result[-3] > best_f1: # -3 index is val F1
                best_f1 = result[-3]
                best_method = method
                best_config = result

    if best_config:
        method, epochs, seed, lr, bs, val_ce, val_acc, val_f1, test_acc, test_f1 = best_config
        print("\n" + "=" * 80)
        print(f"BEST OVERALL CONFIGURATION (Test F1: {test_f1:.4f}):")
        print(f"Fine-tuning method: {best_method}")
        print(f"Learning rate: {lr}")
        print(f"Batch size: {bs}")
        print(f"Epochs: {epochs}")
        print("=" * 80)

## HyperParameter Search of Bert on Full fine tuning

Best hyperparams which we found:
- lr: 5e-5
- epoch:4

In [ ]:
model_name = "bert-base-uncased"


all_results = hyperparameter_search(
            model_name=model_name,
            fine_tuning_methods=["full"],
            learning_rates=[1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3],
            batch_sizes=[16], # as in the paper for this task
            epochs=[3,4, 5]
        )


Running experiments for full fine-tuning on bert-base-uncased
{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}


Token indices sequence length is longer than the specified maximum sequence length for this model (687 > 512). Running this sequence through the model will result in indexing errors


{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.0000], device='cuda:0')
length_dataset 932 232 302

== Experiment 1 of 7:
=> Learning Rate: 1e-06, Batch Size: 16


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: 108,895,493 / 108,895,493 (100.00%)

=== Epoch 1
-- TRAIN Phase


...

train Loss: 1.0541
-- VAL Phase


Val Loop: 100%|██████████| 15/15 [00:02<00:00,  5.41it/s]


val Loss: 1.0220
== Val Cross Entropy:  1.0287615204679554
== Val Accuracy:  0.4891304347826087
== Val F1:  0.3213265629958743
== Early Stopping Count:  0


Testing: 100%|██████████| 19/19 [00:03<00:00,  5.70it/s]


 Test Results — LR: 0.001, BS: 16
 Test Accuracy: 0.5240
 Test F1 Score: 0.3604


In [ ]:
display_results(all_results)


SUMMARY OF RESULTS

FULL FINE-TUNING RESULTS:
------------------------------------------------------------
Method   Epochs LR       BS   Val CE   Val Acc  Val F1   Test Acc Test F1 
------------------------------------------------------------
full     5      0.000020 16   0.7196   0.7717   0.7651   0.8284   0.8221  
full     4      0.000030 16   0.7153   0.7826   0.7674   0.8261   0.8209  
full     5      0.000030 16   0.7065   0.7754   0.7608   0.8215   0.8166  
full     3      0.000050 16   0.6895   0.7717   0.7581   0.8169   0.8108  
full     4      0.000050 16   0.6795   0.8007   0.7829   0.8169   0.8107  
full     4      0.000020 16   0.7173   0.7572   0.7415   0.8169   0.8103  
full     4      0.000100 16   0.6833   0.7572   0.7524   0.8146   0.8089  
full     3      0.000030 16   0.7114   0.7645   0.7461   0.8169   0.8085  
full     3      0.000100 16   0.7247   0.7536   0.7426   0.8101   0.8050  
full     5      0.000100 16   0.6771   0.7826   0.7715   0.8078   0.8047  
full  

## Hyperparameter search for Lora fine tuning

Best hyperparams which we found:
- lr: 1e-3
- epoch: 5

In [ ]:
model_name = "bert-base-uncased"


all_results = hyperparameter_search(
            model_name=model_name,
            fine_tuning_methods=["lora"],
            learning_rates=[1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3],
            batch_sizes=[16], # as in the paper for this task
            epochs=[3,4, 5]
        )


Running experiments for lora fine-tuning on bert-base-uncased
{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}


Token indices sequence length is longer than the specified maximum sequence length for this model (687 > 512). Running this sequence through the model will result in indexing errors


{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.0000], device='cuda:0')
length_dataset 932 232 302

== Experiment 1 of 7:
=> Learning Rate: 1e-06, Batch Size: 16


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: 888,581 / 109,784,074 (0.81%)

=== Epoch 1
-- TRAIN Phase


...

train Loss: 0.2433
-- VAL Phase


Val Loop: 100%|██████████| 15/15 [00:03<00:00,  4.92it/s]


val Loss: 0.7036
== Val Cross Entropy:  0.7058236805465201
== Val Accuracy:  0.7681159420289855
== Val F1:  0.7599274832138881
== Early Stopping Count:  1


Testing: 100%|██████████| 19/19 [00:03<00:00,  5.03it/s]


 Test Results — LR: 0.001, BS: 16
 Test Accuracy: 0.7918
 Test F1 Score: 0.7872


In [ ]:
display_results(all_results)


SUMMARY OF RESULTS

LORA FINE-TUNING RESULTS:
------------------------------------------------------------
Method   Epochs LR       BS   Val CE   Val Acc  Val F1   Test Acc Test F1 
------------------------------------------------------------
lora     5      0.001000 16   0.6918   0.7681   0.7566   0.7918   0.7872  
lora     3      0.001000 16   0.6772   0.7609   0.7387   0.7826   0.7777  
lora     4      0.001000 16   0.6685   0.7645   0.7537   0.7757   0.7640  
lora     5      0.000100 16   0.8124   0.6123   0.5758   0.6636   0.6551  
lora     4      0.000100 16   0.8527   0.5688   0.4948   0.6018   0.5737  
lora     3      0.000100 16   0.8626   0.5471   0.4588   0.5835   0.5321  
lora     5      0.000010 16   1.2771   0.4710   0.4184   0.5698   0.4992  
lora     4      0.000010 16   1.3686   0.3623   0.3585   0.5423   0.4949  
lora     5      0.000050 16   0.8742   0.5290   0.4322   0.5469   0.4484  
lora     3      0.000020 16   1.2069   0.5036   0.4184   0.5423   0.4399  
lora  

## Hyperparameters Search for Adapter fine tuning

Best hyperparams which we found:
- lr: 1e-3
- epoch: 11

In [ ]:
model_name = "bert-base-uncased"


all_results = hyperparameter_search(
            model_name=model_name,
            fine_tuning_methods=["adapter"],
            learning_rates=[1e-6, 1e-5, 2e-5, 3e-5, 5e-5, 1e-4, 1e-3],
            batch_sizes=[16], # as in the paper for this task
            epochs=[6, 9, 11]
        )


Running experiments for adapter fine-tuning on bert-base-uncased
{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}


Token indices sequence length is longer than the specified maximum sequence length for this model (687 > 512). Running this sequence through the model will result in indexing errors


{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.0000], device='cuda:0')
length_dataset 932 232 302

== Experiment 1 of 7:
=> Learning Rate: 1e-06, Batch Size: 16


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: 1,792,901 / 110,684,549 (1.62%)

=== Epoch 1
-- TRAIN Phase


...

train Loss: 0.0205
-- VAL Phase


Val Loop: 100%|██████████| 15/15 [00:03<00:00,  4.88it/s]


val Loss: 1.1657
== Val Cross Entropy:  1.2056357234652186
== Val Accuracy:  0.8043478260869565
== Val F1:  0.7928282813560255
== Early Stopping Count:  3


Testing: 100%|██████████| 19/19 [00:03<00:00,  5.03it/s]


 Test Results — LR: 0.001, BS: 16
 Test Accuracy: 0.8055
 Test F1 Score: 0.8034


In [ ]:
display_results(all_results)


SUMMARY OF RESULTS

ADAPTER FINE-TUNING RESULTS:
------------------------------------------------------------
Method   Epochs LR       BS   Val CE   Val Acc  Val F1   Test Acc Test F1 
------------------------------------------------------------
adapter  6      0.001000 16   0.6203   0.7935   0.7835   0.8375   0.8288  
adapter  11     0.001000 16   0.6339   0.8152   0.8013   0.8055   0.8034  
adapter  9      0.001000 16   0.6674   0.8007   0.7816   0.8032   0.8018  
adapter  9      0.000100 16   0.7514   0.7428   0.7248   0.7529   0.7400  
adapter  11     0.000100 16   0.7445   0.7319   0.7220   0.7391   0.7251  
adapter  11     0.000050 16   0.7881   0.7174   0.6939   0.7094   0.6963  
adapter  6      0.000100 16   0.7811   0.7355   0.7117   0.6865   0.6728  
adapter  9      0.000050 16   0.8046   0.6884   0.6612   0.6613   0.6506  
adapter  11     0.000030 16   0.8283   0.6667   0.6326   0.6339   0.6215  
adapter  6      0.000050 16   0.8395   0.6486   0.6062   0.6362   0.6205  
ada

## Finbert training on best hyperparameters

Finbert full fine tuning

In [ ]:
model_name = "yiyanghkust/finbert-pretrain"
results = train_and_evaluate(
        model_name=model_name,
        fine_tuning_method="full",
        batch_sizes=[16],
        learning_rates=[5e-5],
        num_epochs=4
)


config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.0000], device='cuda:0')
length_dataset 932 232 302

== Experiment 1 of 1:
=> Learning Rate: 5e-05, Batch Size: 16


pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

train Loss: 0.0836
-- VAL Phase


Val Loop: 100%|██████████| 15/15 [00:02<00:00,  5.35it/s]


val Loss: 0.3986
== Val Cross Entropy:  0.410409152507782
== Val Accuracy:  0.8219895287958116
== Val F1:  0.809085655180519
== Early Stopping Count:  0


Testing: 100%|██████████| 19/19 [00:02<00:00,  6.45it/s]


 Test Results — LR: 5e-05, BS: 16
 Test Accuracy: 0.8249
 Test F1 Score: 0.8249


Finbert Lora fine tuning

In [ ]:
model_name = "yiyanghkust/finbert-pretrain"
results = train_and_evaluate(
        model_name=model_name,
        fine_tuning_method="lora",
        batch_sizes=[16],
        learning_rates=[1e-3],
        num_epochs=5
)

{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.0000], device='cuda:0')
length_dataset 932 232 302

== Experiment 1 of 1:
=> Learning Rate: 0.001, Batch Size: 16


Some weights of BertForTokenClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: 888,581 / 110,053,642 (0.81%)

=== Epoch 1
-- TRAIN Phase


Train Loop: 100%|██████████| 59/59 [00:28<00:00,  2.11it/s]


train Loss: 0.9717
-- VAL Phase


...

train Loss: 0.1705
-- VAL Phase


Val Loop: 100%|██████████| 15/15 [00:03<00:00,  4.99it/s]


val Loss: 0.4609
== Val Cross Entropy:  0.47243820028058414
== Val Accuracy:  0.7670157068062827
== Val F1:  0.7496477070068256
== Early Stopping Count:  1


Testing: 100%|██████████| 19/19 [00:03<00:00,  5.83it/s]


 Test Results — LR: 0.001, BS: 16
 Test Accuracy: 0.7843
 Test F1 Score: 0.7776


Finbert Adapter fine tuning

In [ ]:
model_name = "yiyanghkust/finbert-pretrain"
results = train_and_evaluate(
        model_name=model_name,
        fine_tuning_method="adapter",
        batch_sizes=[16],
        learning_rates=[1e-3],
        num_epochs= 11
)

{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.0000], device='cuda:0')
length_dataset 932 232 302

== Experiment 1 of 1:
=> Learning Rate: 0.001, Batch Size: 16


Some weights of BertForTokenClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: 1,792,901 / 110,954,117 (1.62%)

=== Epoch 1
-- TRAIN Phase


Train Loop: 100%|██████████| 59/59 [00:26<00:00,  2.20it/s]


train Loss: 1.0305
-- VAL Phase


...

train Loss: 0.0091
-- VAL Phase


Val Loop: 100%|██████████| 15/15 [00:03<00:00,  4.94it/s]


val Loss: 0.6483
== Val Cross Entropy:  0.669503383033363
== Val Accuracy:  0.837696335078534
== Val F1:  0.8378985083180615
== Early Stopping Count:  0


Testing: 100%|██████████| 19/19 [00:03<00:00,  5.80it/s]


 Test Results — LR: 0.001, BS: 16
 Test Accuracy: 0.7913
 Test F1 Score: 0.7926


## FlangBert training on best Hyperparams

FlangBert full fine tuning

In [ ]:
# SALT-NLP/FLANG-BERT
model_name = "SALT-NLP/FLANG-BERT"
results = train_and_evaluate(
        model_name=model_name,
        fine_tuning_method="full",
        batch_sizes=[16],
        learning_rates=[5e-5],
        num_epochs=4
)

tokenizer_config.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}


Token indices sequence length is longer than the specified maximum sequence length for this model (687 > 512). Running this sequence through the model will result in indexing errors


{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.0000], device='cuda:0')
length_dataset 932 232 302

== Experiment 1 of 1:
=> Learning Rate: 5e-05, Batch Size: 16


...

train Loss: 0.1310
-- VAL Phase


Val Loop: 100%|██████████| 15/15 [00:02<00:00,  5.16it/s]


val Loss: 0.8225
== Val Cross Entropy:  0.8387456948644129
== Val Accuracy:  0.7753623188405797
== Val F1:  0.762172719219965
== Early Stopping Count:  1


Testing: 100%|██████████| 19/19 [00:03<00:00,  5.46it/s]


 Test Results — LR: 5e-05, BS: 16
 Test Accuracy: 0.8261
 Test F1 Score: 0.8191


FlangBert lora fine tuning

In [ ]:
model_name = "SALT-NLP/FLANG-BERT"
results = train_and_evaluate(
        model_name=model_name,
        fine_tuning_method="lora",
        batch_sizes=[16],
        learning_rates=[1e-3],
        num_epochs=5
)

{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}


Token indices sequence length is longer than the specified maximum sequence length for this model (687 > 512). Running this sequence through the model will result in indexing errors


{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.0000], device='cuda:0')
length_dataset 932 232 302

== Experiment 1 of 1:
=> Learning Rate: 0.001, Batch Size: 16


Some weights of BertForTokenClassification were not initialized from the model checkpoint at SALT-NLP/FLANG-BERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: 888,581 / 109,784,074 (0.81%)

=== Epoch 1
-- TRAIN Phase


...

train Loss: 0.2784
-- VAL Phase


Val Loop: 100%|██████████| 15/15 [00:03<00:00,  4.90it/s]


val Loss: 0.7470
== Val Cross Entropy:  0.7537680992792393
== Val Accuracy:  0.7572463768115942
== Val F1:  0.756951381951382
== Early Stopping Count:  1


Testing: 100%|██████████| 19/19 [00:03<00:00,  5.05it/s]


 Test Results — LR: 0.001, BS: 16
 Test Accuracy: 0.8124
 Test F1 Score: 0.8061


FlangBert adapter fine tuning

In [ ]:
model_name = "SALT-NLP/FLANG-BERT"
results = train_and_evaluate(
        model_name=model_name,
        fine_tuning_method="adapter",
        batch_sizes=[16],
        learning_rates=[1e-3],
        num_epochs= 11)

{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}


Token indices sequence length is longer than the specified maximum sequence length for this model (687 > 512). Running this sequence through the model will result in indexing errors


{'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
tensor([0.2500, 0.2500, 0.2500, 0.2500, 0.0000], device='cuda:0')
length_dataset 932 232 302

== Experiment 1 of 1:
=> Learning Rate: 0.001, Batch Size: 16


Some weights of BertForTokenClassification were not initialized from the model checkpoint at SALT-NLP/FLANG-BERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: 1,792,901 / 110,684,549 (1.62%)

=== Epoch 1
-- TRAIN Phase


...

train Loss: 0.0145
-- VAL Phase


Val Loop: 100%|██████████| 15/15 [00:03<00:00,  4.87it/s]


val Loss: 0.9960
== Val Cross Entropy:  1.029274794305193
== Val Accuracy:  0.7717391304347826
== Val F1:  0.7626472273689168
== Early Stopping Count:  3


Testing: 100%|██████████| 19/19 [00:03<00:00,  5.00it/s]


 Test Results — LR: 0.001, BS: 16
 Test Accuracy: 0.8032
 Test F1 Score: 0.7996
